In [4]:
import pandas as pd
import numpy as np
import requests
import re
from bs4 import BeautifulSoup
from time import sleep 
pd.options.display.max_columns = 30

In [87]:
teams = pd.DataFrame(['ANA','ARI','ATL','BAL','BOS','CAL','CHC','CHW','CIN','CLE','COL','DET','FLA','HOU','KCR',
                   'LAA','LAD','MIA','MIL','MIN','MON','NYM','NYY','OAK','PHI','PIT','SDP','SEA','SFG','STL','TBD','TBR','TEX',
                    'TOR','WSN'])
                
teams=teams.values
teams

array([['ANA'],
       ['ARI'],
       ['ATL'],
       ['BAL'],
       ['BOS'],
       ['CAL'],
       ['CHC'],
       ['CHW'],
       ['CIN'],
       ['CLE'],
       ['COL'],
       ['DET'],
       ['FLA'],
       ['HOU'],
       ['KCR'],
       ['LAA'],
       ['LAD'],
       ['MIA'],
       ['MIL'],
       ['MIN'],
       ['MON'],
       ['NYM'],
       ['NYY'],
       ['OAK'],
       ['PHI'],
       ['PIT'],
       ['SDP'],
       ['SEA'],
       ['SFG'],
       ['STL'],
       ['TBD'],
       ['TBR'],
       ['TEX'],
       ['TOR'],
       ['WSN']], dtype=object)

In [100]:
def games():
    team_df = pd.DataFrame(columns=['date', 'box', 'team', 'at', 'opponent', 'w_or_l', 'runs', 'runs_allowed',
                                    'innings', 'WL_record', 'rank', 'gb', 'winning_pitcher', 'losing_pitcher',
                                    'save', 'time', 'd_or_n', 'attendance', 'streak', 'double_header', 'runs_per_game',
                                    'runs_moving_avg', 'runs_allowed_moving_avg', 'win_pct', 'opening_day'])
    for year in range (2000, 2018):
        try:
            df = pd.DataFrame(columns=['date', 'box', 'team', 'at', 'opponent', 'w_or_l', 'runs', 'runs_allowed',
                                       'innings', 'WL_record', 'rank', 'gb', 'winning_pitcher', 'losing_pitcher',
                                       'save', 'time', 'd_or_n', 'attendance', 'streak'])
            urls = requests.get('http://www.baseball-reference.com/teams/' + 'ANA' + '/' + str(year) + '-schedule-scores.shtml')
            soup = BeautifulSoup(urls.text, 'html.parser')
            for game in soup.find('table', {'class':'stats_table'}).find_all('tr'):
                results = []
                for element in game.find_all('td'):
                    results.append(element.text)
                    if len(results) == 19:
                        df.loc[len(df)] = results



            # double header variable for when 2 teams play same day in front of same crowd
            double_header = '('
            df['double_header'] = [1 if double_header in x else 0 for x in df.date]

            # convert date column to datetime
            df.date = df.date.str.replace(r"\(.*\)","")
            date = []
            for day in df.date:
                split = day.split(', ')
                date.append(split[1] + ', ' + str(year))
            df.date = date
            df.date = pd.to_datetime(df.date)

            # shift WL_record, rank, gb, streak and fill in the first value
            df.WL_record = df.WL_record.shift()
            df.WL_record.fillna('0-0', inplace=True)

            for col in ['rank', 'gb', 'streak']:
                df[col] = df[col].shift()
                df[col].fillna('0', inplace=True)

            # rolling means for runs and runs allowed
            df['runs_per_game'] = [0 if x == min(df.date) else df[df.date < x].runs.astype(int).mean() for x in df.date]
            df['runs_moving_avg'] = df.runs.rolling(5).mean().shift()
            df['runs_allowed_moving_avg'] = df.runs_allowed.rolling(5).mean().shift()
            df.runs_allowed_moving_avg.fillna(df.iloc[5].runs_allowed_moving_avg, inplace=True)

            # win_pct of last ten games
            win_pct = [list(df.w_or_l[i-10:i]).count('W') / 10. for i in range(10, len(df))]
            for i in range(10):
                win_pct.insert(0, None)
            df['win_pct'] = win_pct

            # return home games only
            df = df[~df['at'].str.contains('@')].reset_index(drop=True)

            # make dummny variable if game was on opening day or not
            df['opening_day'] = [1 if x == min(df.date) else 0 for x in df.date]

            # concatenate the two dataframes 
            team_df = pd.concat([team_df, df]).reset_index(drop=True)

            #sleep
            sleep(0.5)

        except:
            pass

    # save it to csv
    team_df.to_csv('/Users/andreewerner/Desktop/mlb-game-data-ANA.csv', index=False, encoding='utf-8')
    return team_df
games()

,date,box,team,at,opponent,w_or_l,runs,runs_allowed,innings,WL_record,rank,gb,winning_pitcher,losing_pitcher,save,time,d_or_n,attendance,streak,double_header,runs_per_game,runs_moving_avg,runs_allowed_moving_avg,win_pct,opening_day
0,2000-04-03,boxscore,ANA,,NYY,L,2,3,,0-0,0,0,Hernandez,Hill,Rivera,3:02,N,"42,704",0,0,0.000000,NaN,4.4,NaN,1
1,2000-04-04,boxscore,ANA,,NYY,L,3,5,,0-1,3,1.0,Mendoza,Percival,Rivera,3:20,N,"25,818",-,0,2.000000,NaN,4.4,NaN,0
2,2000-04-05,boxscore,ANA,,NYY,W,12,6,,0-2,4,2.0,Schoeneweis,Cone,,3:01,N,"24,560",--,0,2.500000,NaN,4.4,NaN,0
3,2000-04-07,boxscore,ANA,,BOS,W,7,3,,1-2,4,1.0,Dickson,Schourek,,2:28,N,"30,412",+,0,5.666667,NaN,4.4,NaN,0
4,2000-04-08,boxscore,ANA,,BOS,W,7,5,,2-2,3,1.0,Hill,Rose,Percival,3:11,N,"27,369",++,0,6.000000,NaN,4.4,NaN,0
5,2000-04-09,boxscore,ANA,,BOS,L,2,5,,3-2,1,Tied,Martinez,Bottenfield,,2:44,D,"35,847",+++,0,6.200000,6.2,4.4,NaN,0
6,2000-04-10,boxscore,ANA,,TOR,W,6,0,,3-3,3,1.0,Schoeneweis,Carpenter,,2:22,N,"14,338",-,0,5.500000,6.2,4.8,NaN,0
7,2000-04-11,boxscore,ANA,,TOR,W,5,4,,4-3,2,0.5,Ortiz,Escobar,Percival,3:13,N,"15,229",+,0,5.571429,6.8,3.8,NaN,0
8,2000-04-12,boxscore,ANA,,TOR,L,2,6,,5-3,1,up 0.5,Borbon,Petkovsek,,2:57,N,"16,494",++,0,5.500000,5.4,3.4,NaN,0
9,2000-04-24,boxscore,ANA,,DET,W,10,4,,9-10,2,3.0,Hill,Mlicki,,2:55,N,"16,433",--,0,5.578947,6.6,8.4,0.4,0


In [101]:
def games():
    team_df = pd.DataFrame(columns=['date', 'box', 'team', 'at', 'opponent', 'w_or_l', 'runs', 'runs_allowed',
                                    'innings', 'WL_record', 'rank', 'gb', 'winning_pitcher', 'losing_pitcher',
                                    'save', 'time', 'd_or_n', 'attendance', 'streak', 'double_header', 'runs_per_game',
                                    'runs_moving_avg', 'runs_allowed_moving_avg', 'win_pct', 'opening_day'])

    for year in range (2000, 2018):
        try:
            df = pd.DataFrame(columns=['date', 'box', 'team', 'at', 'opponent', 'w_or_l', 'runs', 'runs_allowed',
                                       'innings', 'WL_record', 'rank', 'gb', 'winning_pitcher', 'losing_pitcher',
                                       'save', 'time', 'd_or_n', 'attendance', 'streak'])
            urls = requests.get('http://www.baseball-reference.com/teams/' + 'ARI' + '/' + str(year) + '-schedule-scores.shtml')
            soup = BeautifulSoup(urls.text, 'html.parser')
            for game in soup.find('table', {'class':'stats_table'}).find_all('tr'):
                results = []
                for element in game.find_all('td'):
                    results.append(element.text)
                    if len(results) == 19:
                        df.loc[len(df)] = results



            # double header variable for when 2 teams play same day in front of same crowd
            double_header = '('
            df['double_header'] = [1 if double_header in x else 0 for x in df.date]

            # convert date column to datetime
            df.date = df.date.str.replace(r"\(.*\)","")
            date = []
            for day in df.date:
                split = day.split(', ')
                date.append(split[1] + ', ' + str(year))
            df.date = date
            df.date = pd.to_datetime(df.date)

            # shift WL_record, rank, gb, streak and fill in the first value
            df.WL_record = df.WL_record.shift()
            df.WL_record.fillna('0-0', inplace=True)

            for col in ['rank', 'gb', 'streak']:
                df[col] = df[col].shift()
                df[col].fillna('0', inplace=True)

            # rolling means for runs and runs allowed
            df['runs_per_game'] = [0 if x == min(df.date) else df[df.date < x].runs.astype(int).mean() for x in df.date]
            df['runs_moving_avg'] = df.runs.rolling(5).mean().shift()
            df['runs_allowed_moving_avg'] = df.runs_allowed.rolling(5).mean().shift()
            df.runs_allowed_moving_avg.fillna(df.iloc[5].runs_allowed_moving_avg, inplace=True)

            # win_pct of last ten games
            win_pct = [list(df.w_or_l[i-10:i]).count('W') / 10. for i in range(10, len(df))]
            for i in range(10):
                win_pct.insert(0, None)
            df['win_pct'] = win_pct

            # return home games only
            df = df[~df['at'].str.contains('@')].reset_index(drop=True)

            # make dummny variable if game was on opening day or not
            df['opening_day'] = [1 if x == min(df.date) else 0 for x in df.date]

            # concatenate the two dataframes 
            team_df = pd.concat([team_df, df]).reset_index(drop=True)

            #sleep
            sleep(0.5)

        except:
            pass

    # save it to csv
    team_df.to_csv('/Users/andreewerner/Desktop/mlb-game-data-ARI.csv', index=False, encoding='utf-8')
    return team_df
games()

,date,box,team,at,opponent,w_or_l,runs,runs_allowed,innings,WL_record,rank,gb,winning_pitcher,losing_pitcher,save,time,d_or_n,attendance,streak,double_header,runs_per_game,runs_moving_avg,runs_allowed_moving_avg,win_pct,opening_day
0,2000-04-04,boxscore,ARI,,PHI,W,6,4,,0-0,0,0,Johnson,Ashby,Holmes,2:24,N,"44,298",0,0,0.000000,NaN,4.2,NaN,1
1,2000-04-05,boxscore,ARI,,PHI,W,11,3,,1-0,2,0.5,Stottlemyre,Byrd,Morgan,2:53,N,"29,291",+,0,6.000000,NaN,4.2,NaN,0
2,2000-04-06,boxscore,ARI,,PHI,W-wo,3,2,11,2-0,1,up 0.5,Springer,Schrenk,,3:03,N,"28,774",++,0,8.500000,NaN,4.2,NaN,0
3,2000-04-07,boxscore,ARI,,PIT,L,2,7,,3-0,1,up 1.0,Christiansen,Reynoso,,2:37,N,"32,536",+++,0,6.666667,NaN,4.2,NaN,0
4,2000-04-08,boxscore,ARI,,PIT,W-wo,6,5,,3-1,1,Tied,Swindell,Christiansen,,2:51,D,"33,298",-,0,5.500000,NaN,4.2,NaN,0
5,2000-04-09,boxscore,ARI,,PIT,W,1,0,,4-1,1,up 1.0,Johnson,Schmidt,,2:21,D,"34,204",+,0,5.600000,5.6,4.2,NaN,0
6,2000-04-17,boxscore,ARI,,COL,L,1,9,,9-3,1,up 1.5,Yoshii,Daal,,2:42,N,"33,743",+++,0,4.666667,3.8,3.2,0.5,0
7,2000-04-18,boxscore,ARI,,COL,W,7,1,,9-4,1,up 1.5,Reynoso,Karl,,2:50,N,"30,275",-,0,4.384615,3.6,4.4,0.5,0
8,2000-04-19,boxscore,ARI,,COL,W,8,7,,10-4,1,up 1.5,Morgan,Tavarez,Kim,3:00,N,"31,237",+,0,4.571429,4.6,3.8,0.6,0
9,2000-04-20,boxscore,ARI,,COL,W,3,0,,11-4,1,up 2.5,Johnson,Arrojo,,2:26,N,"34,458",++,0,4.800000,5.2,4.4,0.7,0


In [102]:
def games():
    team_df = pd.DataFrame(columns=['date', 'box', 'team', 'at', 'opponent', 'w_or_l', 'runs', 'runs_allowed',
                                    'innings', 'WL_record', 'rank', 'gb', 'winning_pitcher', 'losing_pitcher',
                                    'save', 'time', 'd_or_n', 'attendance', 'streak', 'double_header', 'runs_per_game',
                                    'runs_moving_avg', 'runs_allowed_moving_avg', 'win_pct', 'opening_day'])

    for year in range (2000, 2018):
        try:
            df = pd.DataFrame(columns=['date', 'box', 'team', 'at', 'opponent', 'w_or_l', 'runs', 'runs_allowed',
                                       'innings', 'WL_record', 'rank', 'gb', 'winning_pitcher', 'losing_pitcher',
                                       'save', 'time', 'd_or_n', 'attendance', 'streak'])
            urls = requests.get('http://www.baseball-reference.com/teams/' + 'ATL' + '/' + str(year) + '-schedule-scores.shtml')
            soup = BeautifulSoup(urls.text, 'html.parser')
            for game in soup.find('table', {'class':'stats_table'}).find_all('tr'):
                results = []
                for element in game.find_all('td'):
                    results.append(element.text)
                    if len(results) == 19:
                        df.loc[len(df)] = results



            # double header variable for when 2 teams play same day in front of same crowd
            double_header = '('
            df['double_header'] = [1 if double_header in x else 0 for x in df.date]

            # convert date column to datetime
            df.date = df.date.str.replace(r"\(.*\)","")
            date = []
            for day in df.date:
                split = day.split(', ')
                date.append(split[1] + ', ' + str(year))
            df.date = date
            df.date = pd.to_datetime(df.date)

            # shift WL_record, rank, gb, streak and fill in the first value
            df.WL_record = df.WL_record.shift()
            df.WL_record.fillna('0-0', inplace=True)

            for col in ['rank', 'gb', 'streak']:
                df[col] = df[col].shift()
                df[col].fillna('0', inplace=True)

            # rolling means for runs and runs allowed
            df['runs_per_game'] = [0 if x == min(df.date) else df[df.date < x].runs.astype(int).mean() for x in df.date]
            df['runs_moving_avg'] = df.runs.rolling(5).mean().shift()
            df['runs_allowed_moving_avg'] = df.runs_allowed.rolling(5).mean().shift()
            df.runs_allowed_moving_avg.fillna(df.iloc[5].runs_allowed_moving_avg, inplace=True)

            # win_pct of last ten games
            win_pct = [list(df.w_or_l[i-10:i]).count('W') / 10. for i in range(10, len(df))]
            for i in range(10):
                win_pct.insert(0, None)
            df['win_pct'] = win_pct

            # return home games only
            df = df[~df['at'].str.contains('@')].reset_index(drop=True)

            # make dummny variable if game was on opening day or not
            df['opening_day'] = [1 if x == min(df.date) else 0 for x in df.date]

            # concatenate the two dataframes 
            team_df = pd.concat([team_df, df]).reset_index(drop=True)

            #sleep
            sleep(0.5)

        except:
            pass

    # save it to csv
    team_df.to_csv('/Users/andreewerner/Desktop/mlb-game-data-ATL.csv', index=False, encoding='utf-8')
    return team_df
games()

,date,box,team,at,opponent,w_or_l,runs,runs_allowed,innings,WL_record,rank,gb,winning_pitcher,losing_pitcher,save,time,d_or_n,attendance,streak,double_header,runs_per_game,runs_moving_avg,runs_allowed_moving_avg,win_pct,opening_day
0,2000-04-03,boxscore,ATL,,COL,W,2,0,,0-0,0,0,Maddux,Astacio,Remlinger,2:14,D,"42,255",0,0,0.000000,NaN,4.4,NaN,1
1,2000-04-04,boxscore,ATL,,COL,L,3,5,,1-0,1,Tied,Tavarez,Burkett,Jimenez,3:01,N,"26,132",+,0,2.000000,NaN,4.4,NaN,0
2,2000-04-05,boxscore,ATL,,COL,W,9,6,,1-1,2,0.5,Chen,Aybar,Ligtenberg,3:09,N,"30,008",-,0,2.500000,NaN,4.4,NaN,0
3,2000-04-07,boxscore,ATL,,SFG,L,2,6,,2-1,1,up 0.5,Gardner,Mulholland,,2:36,N,"35,343",+,0,4.666667,NaN,4.4,NaN,0
4,2000-04-08,boxscore,ATL,,SFG,W,7,5,,2-2,2,0.5,Maddux,Hernandez,Ligtenberg,2:38,N,"35,938",-,0,4.000000,NaN,4.4,NaN,0
5,2000-04-09,boxscore,ATL,,SFG,W,9,3,,3-2,1,up 0.5,Glavine,Ortiz,,3:09,D,"32,654",+,0,4.600000,4.6,4.4,NaN,0
6,2000-04-18,boxscore,ATL,,PHI,W-wo,4,3,12,6-6,2,0.5,Rivera,Reyes,,3:48,N,"34,903",+,0,4.333333,3.4,4.8,0.5,0
7,2000-04-19,boxscore,ATL,,PHI,W,10,1,,7-6,2,1.0,Glavine,Wolf,,2:59,N,"29,992",++,0,4.307692,3.4,3.2,0.4,0
8,2000-04-20,boxscore,ATL,,PHI,W,6,4,,8-6,1,Tied,Millwood,Aldred,Rocker,2:47,N,"31,451",+++,0,4.714286,5.0,2.8,0.5,0
9,2000-04-21,boxscore,ATL,,PIT,W,6,2,,9-6,1,Tied,Mulholland,Garcia,,2:28,N,"33,790",++++,0,4.800000,5.0,3.0,0.5,0


In [103]:
def games():
    team_df = pd.DataFrame(columns=['date', 'box', 'team', 'at', 'opponent', 'w_or_l', 'runs', 'runs_allowed',
                                    'innings', 'WL_record', 'rank', 'gb', 'winning_pitcher', 'losing_pitcher',
                                    'save', 'time', 'd_or_n', 'attendance', 'streak', 'double_header', 'runs_per_game',
                                    'runs_moving_avg', 'runs_allowed_moving_avg', 'win_pct', 'opening_day'])
   # for team in teams:
    for year in range (2000, 2018):
        try:
            df = pd.DataFrame(columns=['date', 'box', 'team', 'at', 'opponent', 'w_or_l', 'runs', 'runs_allowed',
                                       'innings', 'WL_record', 'rank', 'gb', 'winning_pitcher', 'losing_pitcher',
                                       'save', 'time', 'd_or_n', 'attendance', 'streak'])
            urls = requests.get('http://www.baseball-reference.com/teams/' + 'BAL' + '/' + str(year) + '-schedule-scores.shtml')
            soup = BeautifulSoup(urls.text, 'html.parser')
            for game in soup.find('table', {'class':'stats_table'}).find_all('tr'):
                results = []
                for element in game.find_all('td'):
                    results.append(element.text)
                    if len(results) == 19:
                        df.loc[len(df)] = results



            # double header variable for when 2 teams play same day in front of same crowd
            double_header = '('
            df['double_header'] = [1 if double_header in x else 0 for x in df.date]

            # convert date column to datetime
            df.date = df.date.str.replace(r"\(.*\)","")
            date = []
            for day in df.date:
                split = day.split(', ')
                date.append(split[1] + ', ' + str(year))
            df.date = date
            df.date = pd.to_datetime(df.date)

            # shift WL_record, rank, gb, streak and fill in the first value
            df.WL_record = df.WL_record.shift()
            df.WL_record.fillna('0-0', inplace=True)

            for col in ['rank', 'gb', 'streak']:
                df[col] = df[col].shift()
                df[col].fillna('0', inplace=True)

            # rolling means for runs and runs allowed
            df['runs_per_game'] = [0 if x == min(df.date) else df[df.date < x].runs.astype(int).mean() for x in df.date]
            df['runs_moving_avg'] = df.runs.rolling(5).mean().shift()
            df['runs_allowed_moving_avg'] = df.runs_allowed.rolling(5).mean().shift()
            df.runs_allowed_moving_avg.fillna(df.iloc[5].runs_allowed_moving_avg, inplace=True)

            # win_pct of last ten games
            win_pct = [list(df.w_or_l[i-10:i]).count('W') / 10. for i in range(10, len(df))]
            for i in range(10):
                win_pct.insert(0, None)
            df['win_pct'] = win_pct

            # return home games only
            df = df[~df['at'].str.contains('@')].reset_index(drop=True)

            # make dummny variable if game was on opening day or not
            df['opening_day'] = [1 if x == min(df.date) else 0 for x in df.date]

            # concatenate the two dataframes 
            team_df = pd.concat([team_df, df]).reset_index(drop=True)

            #sleep
            sleep(0.5)

        except:
            pass

    # save it to csv
    team_df.to_csv('/Users/andreewerner/Desktop/mlb-game-data-BAL.csv', index=False, encoding='utf-8')
    return team_df
games()

,date,box,team,at,opponent,w_or_l,runs,runs_allowed,innings,WL_record,rank,gb,winning_pitcher,losing_pitcher,save,time,d_or_n,attendance,streak,double_header,runs_per_game,runs_moving_avg,runs_allowed_moving_avg,win_pct,opening_day
0,2000-04-03,boxscore,BAL,,CLE,L,1,4,,0-0,0,0,Colon,Mussina,Karsay,3:07,D,"46,902",0,0,0.000000,NaN,4.8,NaN,1
1,2000-04-05,boxscore,BAL,,CLE,W,11,7,,0-1,5,1.0,Ryan,Kamieniecki,,3:17,N,"33,833",-,0,1.000000,NaN,4.8,NaN,0
2,2000-04-06,boxscore,BAL,,CLE,W,6,2,,1-1,3,0.5,Rapp,Nagy,Groom,2:36,N,"35,181",+,0,6.000000,NaN,4.8,NaN,0
3,2000-04-07,boxscore,BAL,,DET,W,14,10,,2-1,1,Tied,Worrell,Mlicki,,3:33,N,"39,415",++,0,6.000000,NaN,4.8,NaN,0
4,2000-04-08,boxscore,BAL,,DET,W-wo,2,1,10,3-1,1,up 1.0,Reyes,Brocail,,3:06,D,"42,547",+++,0,8.000000,NaN,4.8,NaN,0
5,2000-04-09,boxscore,BAL,,DET,W,11,6,,4-1,1,up 1.0,Mercedes,Nitkowski,,3:15,D,"42,178",++++,0,6.800000,6.8,4.8,NaN,0
6,2000-04-19,boxscore,BAL,,TBD,W-wo,3,2,,7-5,2,1.5,Trombley,Mecir,,2:49,N,"40,077",++,0,6.750000,6.2,5.4,0.5,0
7,2000-04-20,boxscore,BAL,,TBD,W,8,4,,8-5,2,2.5,Rapp,Rupe,,2:41,D,"34,536",+++,0,6.461538,5.6,4.4,0.4,0
8,2000-04-28,boxscore,BAL,,TEX,W-wo,4,3,,12-9,3,1.5,Groom,Zimmerman,,2:58,N,"44,092",--,0,6.333333,5.2,8.2,0.5,0
9,2000-04-29,boxscore,BAL,,TEX,W,3,1,,13-9,2,1.5,Mussina,Oliver,,2:46,D,"48,230",+,0,6.227273,5.6,8.2,0.4,0


In [1]:
def games():
    team_df = pd.DataFrame(columns=['date', 'box', 'team', 'at', 'opponent', 'w_or_l', 'runs', 'runs_allowed',
                                    'innings', 'WL_record', 'rank', 'gb', 'winning_pitcher', 'losing_pitcher',
                                    'save', 'time', 'd_or_n', 'attendance', 'streak', 'double_header', 'runs_per_game',
                                    'runs_moving_avg', 'runs_allowed_moving_avg', 'win_pct', 'opening_day'])
   # for team in teams:
    for year in range (2000, 2018):
        try:
            df = pd.DataFrame(columns=['date', 'box', 'team', 'at', 'opponent', 'w_or_l', 'runs', 'runs_allowed',
                                       'innings', 'WL_record', 'rank', 'gb', 'winning_pitcher', 'losing_pitcher',
                                       'save', 'time', 'd_or_n', 'attendance', 'streak'])
            urls = requests.get('http://www.baseball-reference.com/teams/' + 'BOS' + '/' + str(year) + '-schedule-scores.shtml')
            soup = BeautifulSoup(urls.text, 'html.parser')
            for game in soup.find('table', {'class':'stats_table'}).find_all('tr'):
                results = []
                for element in game.find_all('td'):
                    results.append(element.text)
                    if len(results) == 19:
                        df.loc[len(df)] = results



            # double header variable for when 2 teams play same day in front of same crowd
            double_header = '('
            df['double_header'] = [1 if double_header in x else 0 for x in df.date]

            # convert date column to datetime
            df.date = df.date.str.replace(r"\(.*\)","")
            date = []
            for day in df.date:
                split = day.split(', ')
                date.append(split[1] + ', ' + str(year))
            df.date = date
            df.date = pd.to_datetime(df.date)

            # shift WL_record, rank, gb, streak and fill in the first value
            df.WL_record = df.WL_record.shift()
            df.WL_record.fillna('0-0', inplace=True)

            for col in ['rank', 'gb', 'streak']:
                df[col] = df[col].shift()
                df[col].fillna('0', inplace=True)

            # rolling means for runs and runs allowed
            df['runs_per_game'] = [0 if x == min(df.date) else df[df.date < x].runs.astype(int).mean() for x in df.date]
            df['runs_moving_avg'] = df.runs.rolling(5).mean().shift()
            df['runs_allowed_moving_avg'] = df.runs_allowed.rolling(5).mean().shift()
            df.runs_allowed_moving_avg.fillna(df.iloc[5].runs_allowed_moving_avg, inplace=True)

            # win_pct of last ten games
            win_pct = [list(df.w_or_l[i-10:i]).count('W') / 10. for i in range(10, len(df))]
            for i in range(10):
                win_pct.insert(0, None)
            df['win_pct'] = win_pct

            # return home games only
            df = df[~df['at'].str.contains('@')].reset_index(drop=True)

            # make dummny variable if game was on opening day or not
            df['opening_day'] = [1 if x == min(df.date) else 0 for x in df.date]

            # concatenate the two dataframes 
            team_df = pd.concat([team_df, df]).reset_index(drop=True)

            #sleep
            sleep(0.5)

        except:
            pass

    # save it to csv
    team_df.to_csv('/Users/andreewerner/Desktop/mlb-game-data-BOS.csv', index=False, encoding='utf-8')
    return team_df
games()

NameError: name 'pd' is not defined

In [105]:
def games():
    team_df = pd.DataFrame(columns=['date', 'box', 'team', 'at', 'opponent', 'w_or_l', 'runs', 'runs_allowed',
                                    'innings', 'WL_record', 'rank', 'gb', 'winning_pitcher', 'losing_pitcher',
                                    'save', 'time', 'd_or_n', 'attendance', 'streak', 'double_header', 'runs_per_game',
                                    'runs_moving_avg', 'runs_allowed_moving_avg', 'win_pct', 'opening_day'])
   # for team in teams:
    for year in range (2000, 2018):
        try:
            df = pd.DataFrame(columns=['date', 'box', 'team', 'at', 'opponent', 'w_or_l', 'runs', 'runs_allowed',
                                       'innings', 'WL_record', 'rank', 'gb', 'winning_pitcher', 'losing_pitcher',
                                       'save', 'time', 'd_or_n', 'attendance', 'streak'])
            urls = requests.get('http://www.baseball-reference.com/teams/' + 'CAL' + '/' + str(year) + '-schedule-scores.shtml')
            soup = BeautifulSoup(urls.text, 'html.parser')
            for game in soup.find('table', {'class':'stats_table'}).find_all('tr'):
                results = []
                for element in game.find_all('td'):
                    results.append(element.text)
                    if len(results) == 19:
                        df.loc[len(df)] = results



            # double header variable for when 2 teams play same day in front of same crowd
            double_header = '('
            df['double_header'] = [1 if double_header in x else 0 for x in df.date]

            # convert date column to datetime
            df.date = df.date.str.replace(r"\(.*\)","")
            date = []
            for day in df.date:
                split = day.split(', ')
                date.append(split[1] + ', ' + str(year))
            df.date = date
            df.date = pd.to_datetime(df.date)

            # shift WL_record, rank, gb, streak and fill in the first value
            df.WL_record = df.WL_record.shift()
            df.WL_record.fillna('0-0', inplace=True)

            for col in ['rank', 'gb', 'streak']:
                df[col] = df[col].shift()
                df[col].fillna('0', inplace=True)

            # rolling means for runs and runs allowed
            df['runs_per_game'] = [0 if x == min(df.date) else df[df.date < x].runs.astype(int).mean() for x in df.date]
            df['runs_moving_avg'] = df.runs.rolling(5).mean().shift()
            df['runs_allowed_moving_avg'] = df.runs_allowed.rolling(5).mean().shift()
            df.runs_allowed_moving_avg.fillna(df.iloc[5].runs_allowed_moving_avg, inplace=True)

            # win_pct of last ten games
            win_pct = [list(df.w_or_l[i-10:i]).count('W') / 10. for i in range(10, len(df))]
            for i in range(10):
                win_pct.insert(0, None)
            df['win_pct'] = win_pct

            # return home games only
            df = df[~df['at'].str.contains('@')].reset_index(drop=True)

            # make dummny variable if game was on opening day or not
            df['opening_day'] = [1 if x == min(df.date) else 0 for x in df.date]

            # concatenate the two dataframes 
            team_df = pd.concat([team_df, df]).reset_index(drop=True)

            #sleep
            sleep(0.5)

        except:
            pass

    # save it to csv
    team_df.to_csv('/Users/andreewerner/Desktop/mlb-game-data-CAL.csv', index=False, encoding='utf-8')
    return team_df
games()

,date,box,team,at,opponent,w_or_l,runs,runs_allowed,innings,WL_record,rank,gb,winning_pitcher,losing_pitcher,save,time,d_or_n,attendance,streak,double_header,runs_per_game,runs_moving_avg,runs_allowed_moving_avg,win_pct,opening_day


In [106]:
def games():
    team_df = pd.DataFrame(columns=['date', 'box', 'team', 'at', 'opponent', 'w_or_l', 'runs', 'runs_allowed',
                                    'innings', 'WL_record', 'rank', 'gb', 'winning_pitcher', 'losing_pitcher',
                                    'save', 'time', 'd_or_n', 'attendance', 'streak', 'double_header', 'runs_per_game',
                                    'runs_moving_avg', 'runs_allowed_moving_avg', 'win_pct', 'opening_day'])
   # for team in teams:
    for year in range (2000, 2018):
        try:
            df = pd.DataFrame(columns=['date', 'box', 'team', 'at', 'opponent', 'w_or_l', 'runs', 'runs_allowed',
                                       'innings', 'WL_record', 'rank', 'gb', 'winning_pitcher', 'losing_pitcher',
                                       'save', 'time', 'd_or_n', 'attendance', 'streak'])
            urls = requests.get('http://www.baseball-reference.com/teams/' + 'CHC' + '/' + str(year) + '-schedule-scores.shtml')
            soup = BeautifulSoup(urls.text, 'html.parser')
            for game in soup.find('table', {'class':'stats_table'}).find_all('tr'):
                results = []
                for element in game.find_all('td'):
                    results.append(element.text)
                    if len(results) == 19:
                        df.loc[len(df)] = results



            # double header variable for when 2 teams play same day in front of same crowd
            double_header = '('
            df['double_header'] = [1 if double_header in x else 0 for x in df.date]

            # convert date column to datetime
            df.date = df.date.str.replace(r"\(.*\)","")
            date = []
            for day in df.date:
                split = day.split(', ')
                date.append(split[1] + ', ' + str(year))
            df.date = date
            df.date = pd.to_datetime(df.date)

            # shift WL_record, rank, gb, streak and fill in the first value
            df.WL_record = df.WL_record.shift()
            df.WL_record.fillna('0-0', inplace=True)

            for col in ['rank', 'gb', 'streak']:
                df[col] = df[col].shift()
                df[col].fillna('0', inplace=True)

            # rolling means for runs and runs allowed
            df['runs_per_game'] = [0 if x == min(df.date) else df[df.date < x].runs.astype(int).mean() for x in df.date]
            df['runs_moving_avg'] = df.runs.rolling(5).mean().shift()
            df['runs_allowed_moving_avg'] = df.runs_allowed.rolling(5).mean().shift()
            df.runs_allowed_moving_avg.fillna(df.iloc[5].runs_allowed_moving_avg, inplace=True)

            # win_pct of last ten games
            win_pct = [list(df.w_or_l[i-10:i]).count('W') / 10. for i in range(10, len(df))]
            for i in range(10):
                win_pct.insert(0, None)
            df['win_pct'] = win_pct

            # return home games only
            df = df[~df['at'].str.contains('@')].reset_index(drop=True)

            # make dummny variable if game was on opening day or not
            df['opening_day'] = [1 if x == min(df.date) else 0 for x in df.date]

            # concatenate the two dataframes 
            team_df = pd.concat([team_df, df]).reset_index(drop=True)

            #sleep
            sleep(0.5)

        except:
            pass

    # save it to csv
    team_df.to_csv('/Users/andreewerner/Desktop/mlb-game-data-CHC.csv', index=False, encoding='utf-8')
    return team_df
games()

,date,box,team,at,opponent,w_or_l,runs,runs_allowed,innings,WL_record,rank,gb,winning_pitcher,losing_pitcher,save,time,d_or_n,attendance,streak,double_header,runs_per_game,runs_moving_avg,runs_allowed_moving_avg,win_pct,opening_day
0,2000-03-30,boxscore,CHC,,NYM,L,1,5,11,1-0,1,up 0.5,Cook,Young,,3:55,N,"55,000",+,0,5.000000,NaN,7.6,NaN,1
1,2000-04-10,boxscore,CHC,,ATL,W-wo,4,3,,2-6,6,4.0,Guthrie,Ligtenberg,,2:28,D,"38,655",--,0,4.250000,5.4,8.2,NaN,0
2,2000-04-12,boxscore,CHC,,ATL,W,11,4,,3-6,6,4.0,Farnsworth,Mulholland,,2:40,D,"26,838",+,0,4.222222,5.4,6.8,NaN,0
3,2000-04-13,boxscore,CHC,,ATL,W-wo,3,2,,4-6,5,3.5,Guthrie,Remlinger,,2:24,D,"20,152",++,0,4.900000,7.0,5.0,0.3,0
4,2000-04-14,boxscore,CHC,,FLA,L,4,9,,5-6,3,2.5,Fernandez,Lorraine,,2:55,D,"21,324",+++,0,4.727273,5.6,4.2,0.2,0
5,2000-04-15,boxscore,CHC,,FLA,W,4,2,,5-7,4,2.5,Downs,Dempster,Aguilera,2:33,D,"37,976",-,0,4.666667,5.8,5.2,0.2,0
6,2000-04-16,boxscore,CHC,,FLA,L,5,6,10,6-7,3,2.0,Miceli,Quevedo,Alfonseca,3:12,D,"27,769",+,0,4.615385,5.2,4.0,0.3,0
7,2000-04-17,boxscore,CHC,,FLA,L,5,6,,6-8,4,2.5,Sanchez,Farnsworth,Alfonseca,2:51,D,"17,572",-,0,4.642857,5.4,4.6,0.3,0
8,2000-04-28,boxscore,CHC,,ARI,W,6,5,,9-15,5,6.0,Tapani,Springer,Aguilera,2:54,D,"27,507",++,0,5.625000,9.2,8.8,0.3,0
9,2000-04-29,boxscore,CHC,,ARI,L,4,7,10,10-15,4,6.0,Mantei,Guthrie,Morgan,3:34,D,"37,484",+++,0,5.640000,9.2,8.4,0.4,0


In [108]:
def games():
    team_df = pd.DataFrame(columns=['date', 'box', 'team', 'at', 'opponent', 'w_or_l', 'runs', 'runs_allowed',
                                    'innings', 'WL_record', 'rank', 'gb', 'winning_pitcher', 'losing_pitcher',
                                    'save', 'time', 'd_or_n', 'attendance', 'streak', 'double_header', 'runs_per_game',
                                    'runs_moving_avg', 'runs_allowed_moving_avg', 'win_pct', 'opening_day'])
   # for team in teams:
    for year in range (2000, 2018):
        try:
            df = pd.DataFrame(columns=['date', 'box', 'team', 'at', 'opponent', 'w_or_l', 'runs', 'runs_allowed',
                                       'innings', 'WL_record', 'rank', 'gb', 'winning_pitcher', 'losing_pitcher',
                                       'save', 'time', 'd_or_n', 'attendance', 'streak'])
            urls = requests.get('http://www.baseball-reference.com/teams/' + 'CHW' + '/' + str(year) + '-schedule-scores.shtml')
            soup = BeautifulSoup(urls.text, 'html.parser')
            for game in soup.find('table', {'class':'stats_table'}).find_all('tr'):
                results = []
                for element in game.find_all('td'):
                    results.append(element.text)
                    if len(results) == 19:
                        df.loc[len(df)] = results



            # double header variable for when 2 teams play same day in front of same crowd
            double_header = '('
            df['double_header'] = [1 if double_header in x else 0 for x in df.date]

            # convert date column to datetime
            df.date = df.date.str.replace(r"\(.*\)","")
            date = []
            for day in df.date:
                split = day.split(', ')
                date.append(split[1] + ', ' + str(year))
            df.date = date
            df.date = pd.to_datetime(df.date)

            # shift WL_record, rank, gb, streak and fill in the first value
            df.WL_record = df.WL_record.shift()
            df.WL_record.fillna('0-0', inplace=True)

            for col in ['rank', 'gb', 'streak']:
                df[col] = df[col].shift()
                df[col].fillna('0', inplace=True)

            # rolling means for runs and runs allowed
            df['runs_per_game'] = [0 if x == min(df.date) else df[df.date < x].runs.astype(int).mean() for x in df.date]
            df['runs_moving_avg'] = df.runs.rolling(5).mean().shift()
            df['runs_allowed_moving_avg'] = df.runs_allowed.rolling(5).mean().shift()
            df.runs_allowed_moving_avg.fillna(df.iloc[5].runs_allowed_moving_avg, inplace=True)

            # win_pct of last ten games
            win_pct = [list(df.w_or_l[i-10:i]).count('W') / 10. for i in range(10, len(df))]
            for i in range(10):
                win_pct.insert(0, None)
            df['win_pct'] = win_pct

            # return home games only
            df = df[~df['at'].str.contains('@')].reset_index(drop=True)

            # make dummny variable if game was on opening day or not
            df['opening_day'] = [1 if x == min(df.date) else 0 for x in df.date]

            # concatenate the two dataframes 
            team_df = pd.concat([team_df, df]).reset_index(drop=True)

            #sleep
            sleep(0.5)

        except:
            pass

    # save it to csv
    team_df.to_csv('/Users/andreewerner/Desktop/mlb-game-data-CHW.csv', index=False, encoding='utf-8')
    return team_df
games()

,date,box,team,at,opponent,w_or_l,runs,runs_allowed,innings,WL_record,rank,gb,winning_pitcher,losing_pitcher,save,time,d_or_n,attendance,streak,double_header,runs_per_game,runs_moving_avg,runs_allowed_moving_avg,win_pct,opening_day
0,2000-04-14,boxscore,CHW,,ANA,W,9,4,,6-4,3,1.5,Sirotka,Hill,,3:25,N,"38,912",-,0,7.100000,6.8,6.0,0.6,1
1,2000-04-15,boxscore,CHW,,ANA,L,1,3,,7-4,3,0.5,Bottenfield,Wells,Percival,2:51,D,"14,135",+,0,7.272727,7.2,6.2,0.7,0
2,2000-04-16,boxscore,CHW,,ANA,L,1,3,,7-5,3,0.5,Schoeneweis,Parque,Percival,2:35,D,"10,929",-,0,6.750000,7.0,4.0,0.7,0
3,2000-04-18,boxscore,CHW,,SEA,W,18,11,,7-6,3,1.5,Sturtze,Sele,,3:32,N,"9,898",--,0,6.307692,4.6,3.4,0.6,0
4,2000-04-19,boxscore,CHW,,SEA,W,5,2,,8-6,2,0.5,Lowe,Meche,Howry,2:57,D,"8,425",+,0,7.142857,6.8,5.4,0.6,0
5,2000-04-21,boxscore,CHW,,DET,W,7,2,,9-6,1,up 0.5,Wells,Nitkowski,,2:52,N,"11,278",++,0,7.000000,6.8,4.6,0.6,0
6,2000-04-22,boxscore,CHW,,DET,W,14,6,,10-6,1,up 0.5,Parque,Weaver,,3:19,D,"16,410",+++,0,7.000000,6.4,4.2,0.6,0
7,2000-04-23,boxscore,CHW,,DET,W,9,4,,11-6,1,up 1.0,Baldwin,Borkowski,,2:47,D,"12,154",++++,0,7.411765,9.0,4.8,0.7,0
8,2000-04-24,boxscore,CHW,,BAL,W,8,2,,12-6,1,up 1.5,Eldred,Mussina,,2:14,N,"15,461",+++++,0,7.500000,10.6,5.0,0.7,0
9,2000-04-25,boxscore,CHW,,BAL,L,6,12,,13-6,1,up 1.5,Rapp,Sirotka,,3:15,N,"11,658",++++++,0,7.526316,8.6,3.2,0.7,0


In [109]:
def games():
    team_df = pd.DataFrame(columns=['date', 'box', 'team', 'at', 'opponent', 'w_or_l', 'runs', 'runs_allowed',
                                    'innings', 'WL_record', 'rank', 'gb', 'winning_pitcher', 'losing_pitcher',
                                    'save', 'time', 'd_or_n', 'attendance', 'streak', 'double_header', 'runs_per_game',
                                    'runs_moving_avg', 'runs_allowed_moving_avg', 'win_pct', 'opening_day'])
   # for team in teams:
    for year in range (2000, 2018):
        try:
            df = pd.DataFrame(columns=['date', 'box', 'team', 'at', 'opponent', 'w_or_l', 'runs', 'runs_allowed',
                                       'innings', 'WL_record', 'rank', 'gb', 'winning_pitcher', 'losing_pitcher',
                                       'save', 'time', 'd_or_n', 'attendance', 'streak'])
            urls = requests.get('http://www.baseball-reference.com/teams/' + 'CIN' + '/' + str(year) + '-schedule-scores.shtml')
            soup = BeautifulSoup(urls.text, 'html.parser')
            for game in soup.find('table', {'class':'stats_table'}).find_all('tr'):
                results = []
                for element in game.find_all('td'):
                    results.append(element.text)
                    if len(results) == 19:
                        df.loc[len(df)] = results



            # double header variable for when 2 teams play same day in front of same crowd
            double_header = '('
            df['double_header'] = [1 if double_header in x else 0 for x in df.date]

            # convert date column to datetime
            df.date = df.date.str.replace(r"\(.*\)","")
            date = []
            for day in df.date:
                split = day.split(', ')
                date.append(split[1] + ', ' + str(year))
            df.date = date
            df.date = pd.to_datetime(df.date)

            # shift WL_record, rank, gb, streak and fill in the first value
            df.WL_record = df.WL_record.shift()
            df.WL_record.fillna('0-0', inplace=True)

            for col in ['rank', 'gb', 'streak']:
                df[col] = df[col].shift()
                df[col].fillna('0', inplace=True)

            # rolling means for runs and runs allowed
            df['runs_per_game'] = [0 if x == min(df.date) else df[df.date < x].runs.astype(int).mean() for x in df.date]
            df['runs_moving_avg'] = df.runs.rolling(5).mean().shift()
            df['runs_allowed_moving_avg'] = df.runs_allowed.rolling(5).mean().shift()
            df.runs_allowed_moving_avg.fillna(df.iloc[5].runs_allowed_moving_avg, inplace=True)

            # win_pct of last ten games
            win_pct = [list(df.w_or_l[i-10:i]).count('W') / 10. for i in range(10, len(df))]
            for i in range(10):
                win_pct.insert(0, None)
            df['win_pct'] = win_pct

            # return home games only
            df = df[~df['at'].str.contains('@')].reset_index(drop=True)

            # make dummny variable if game was on opening day or not
            df['opening_day'] = [1 if x == min(df.date) else 0 for x in df.date]

            # concatenate the two dataframes 
            team_df = pd.concat([team_df, df]).reset_index(drop=True)

            #sleep
            sleep(0.5)

        except:
            pass

    # save it to csv
    team_df.to_csv('/Users/andreewerner/Desktop/mlb-game-data-CIN.csv', index=False, encoding='utf-8')
    return team_df
games()

,date,box,team,at,opponent,w_or_l,runs,runs_allowed,innings,WL_record,rank,gb,winning_pitcher,losing_pitcher,save,time,d_or_n,attendance,streak,double_header,runs_per_game,runs_moving_avg,runs_allowed_moving_avg,win_pct,opening_day
0,2000-04-03,boxscore,CIN,,MIL,T,3,3,6,0-0,0,0,,,,1:51,D,"55,596",0,0,0.000000,NaN,5.4,NaN,1
1,2000-04-04,boxscore,CIN,,MIL,L,1,5,,0-0,2,0.5,Bruske,Williamson,,3:11,N,"16,761",,0,3.000000,NaN,5.4,NaN,0
2,2000-04-05,boxscore,CIN,,MIL,L,5,8,,0-1,4,1.0,Haynes,Parris,Wickman,3:08,N,"23,059",-,0,2.000000,NaN,5.4,NaN,0
3,2000-04-06,boxscore,CIN,,MIL,W,5,1,,0-2,4,2.0,Villone,Navarro,,2:53,N,"20,909",--,0,3.000000,NaN,5.4,NaN,0
4,2000-04-07,boxscore,CIN,,CHC,L,6,10,,1-2,4,2.0,Lorraine,Harnisch,Aguilera,2:58,N,"36,383",+,0,3.500000,NaN,5.4,NaN,0
5,2000-04-08,boxscore,CIN,,CHC,W-wo,4,3,11,1-3,5,2.0,Graves,Guthrie,,3:41,D,"37,151",-,0,4.000000,4.0,5.4,NaN,0
6,2000-04-09,boxscore,CIN,,CHC,W-wo,8,7,11,2-3,4,2.0,Graves,Karchner,,4:10,D,"37,798",+,0,4.000000,4.2,5.4,NaN,0
7,2000-04-18,boxscore,CIN,,SFG,L,9,13,,6-6,2,1.5,Rueter,Williamson,,3:51,N,"25,888",++,0,4.846154,5.2,5.0,0.4,0
8,2000-04-19,boxscore,CIN,,SFG,W,5,4,,6-7,2,2.5,Bell,Estes,Graves,3:10,N,"22,795",-,0,5.142857,5.0,7.0,0.3,0
9,2000-04-20,boxscore,CIN,,SFG,W,11,1,,7-7,2,2.5,Neagle,Hernandez,,2:43,D,"27,003",+,0,5.133333,5.0,6.4,0.4,0


In [111]:
def games():
    team_df = pd.DataFrame(columns=['date', 'box', 'team', 'at', 'opponent', 'w_or_l', 'runs', 'runs_allowed',
                                    'innings', 'WL_record', 'rank', 'gb', 'winning_pitcher', 'losing_pitcher',
                                    'save', 'time', 'd_or_n', 'attendance', 'streak', 'double_header', 'runs_per_game',
                                    'runs_moving_avg', 'runs_allowed_moving_avg', 'win_pct', 'opening_day'])
   # for team in teams:
    for year in range (2000, 2018):
        try:
            df = pd.DataFrame(columns=['date', 'box', 'team', 'at', 'opponent', 'w_or_l', 'runs', 'runs_allowed',
                                       'innings', 'WL_record', 'rank', 'gb', 'winning_pitcher', 'losing_pitcher',
                                       'save', 'time', 'd_or_n', 'attendance', 'streak'])
            urls = requests.get('http://www.baseball-reference.com/teams/' + 'CLE' + '/' + str(year) + '-schedule-scores.shtml')
            soup = BeautifulSoup(urls.text, 'html.parser')
            for game in soup.find('table', {'class':'stats_table'}).find_all('tr'):
                results = []
                for element in game.find_all('td'):
                    results.append(element.text)
                    if len(results) == 19:
                        df.loc[len(df)] = results



            # double header variable for when 2 teams play same day in front of same crowd
            double_header = '('
            df['double_header'] = [1 if double_header in x else 0 for x in df.date]

            # convert date column to datetime
            df.date = df.date.str.replace(r"\(.*\)","")
            date = []
            for day in df.date:
                split = day.split(', ')
                date.append(split[1] + ', ' + str(year))
            df.date = date
            df.date = pd.to_datetime(df.date)

            # shift WL_record, rank, gb, streak and fill in the first value
            df.WL_record = df.WL_record.shift()
            df.WL_record.fillna('0-0', inplace=True)

            for col in ['rank', 'gb', 'streak']:
                df[col] = df[col].shift()
                df[col].fillna('0', inplace=True)

            # rolling means for runs and runs allowed
            df['runs_per_game'] = [0 if x == min(df.date) else df[df.date < x].runs.astype(int).mean() for x in df.date]
            df['runs_moving_avg'] = df.runs.rolling(5).mean().shift()
            df['runs_allowed_moving_avg'] = df.runs_allowed.rolling(5).mean().shift()
            df.runs_allowed_moving_avg.fillna(df.iloc[5].runs_allowed_moving_avg, inplace=True)

            # win_pct of last ten games
            win_pct = [list(df.w_or_l[i-10:i]).count('W') / 10. for i in range(10, len(df))]
            for i in range(10):
                win_pct.insert(0, None)
            df['win_pct'] = win_pct

            # return home games only
            df = df[~df['at'].str.contains('@')].reset_index(drop=True)

            # make dummny variable if game was on opening day or not
            df['opening_day'] = [1 if x == min(df.date) else 0 for x in df.date]

            # concatenate the two dataframes 
            team_df = pd.concat([team_df, df]).reset_index(drop=True)

            #sleep
            sleep(0.5)

        except:
            pass

    # save it to csv
    team_df.to_csv('/Users/andreewerner/Desktop/mlb-game-data-CLE.csv', index=False, encoding='utf-8')
    return team_df
games()

,date,box,team,at,opponent,w_or_l,runs,runs_allowed,innings,WL_record,rank,gb,winning_pitcher,losing_pitcher,save,time,d_or_n,attendance,streak,double_header,runs_per_game,runs_moving_avg,runs_allowed_moving_avg,win_pct,opening_day
0,2000-04-14,boxscore,CLE,,TEX,L,2,7,,7-2,1,up 0.5,Helling,Burba,,3:05,D,"42,727",++++++,0,7.666667,8.4,2.6,NaN,1
1,2000-04-15,boxscore,CLE,,TEX,L,4,6,,7-3,1,Tied,Clark,Colon,Wetteland,3:03,D,"40,543",-,0,7.100000,7.6,3.2,0.7,0
2,2000-04-16,boxscore,CLE,,TEX,W-wo,2,1,,7-4,1,Tied,Finley,Wetteland,,2:41,D,"40,626",--,0,6.818182,5.0,3.6,0.6,0
3,2000-04-18,boxscore,CLE,,OAK,L,5,8,,8-4,1,up 1.0,Mulder,Nagy,Isringhausen,3:07,N,"40,551",+,0,6.416667,3.6,3.0,0.6,0
4,2000-04-19,boxscore,CLE,,OAK,L,5,10,,8-5,1,up 0.5,Appier,Wright,,3:46,N,"40,531",-,0,6.307692,3.6,4.4,0.6,0
5,2000-04-20,boxscore,CLE,,OAK,W,9,5,,8-6,2,0.5,Burba,Hudson,,3:31,N,"42,134",--,0,6.214286,3.6,6.4,0.5,0
6,2000-04-28,boxscore,CLE,,BOS,W,4,3,,11-7,2,1.5,Burba,Schourek,Karsay,2:54,N,"42,885",+,0,6.222222,6.0,5.2,0.4,0
7,2000-04-29,boxscore,CLE,,BOS,W,3,2,,12-7,2,2.0,Finley,Martinez,Karsay,2:39,D,"42,453",++,0,6.105263,5.8,3.8,0.4,0
8,2000-04-30,boxscore,CLE,,BOS,L,1,2,,13-7,2,2.0,Martinez,Nagy,Lowe,2:50,D,"42,065",+++,0,5.950000,4.6,3.2,0.5,0
9,2000-05-01,boxscore,CLE,,NYY,L,1,2,,13-8,2,2.0,Mendoza,Wright,Rivera,3:15,N,"42,711",-,0,5.714286,3.6,3.6,0.5,0


In [112]:
def games():
    team_df = pd.DataFrame(columns=['date', 'box', 'team', 'at', 'opponent', 'w_or_l', 'runs', 'runs_allowed',
                                    'innings', 'WL_record', 'rank', 'gb', 'winning_pitcher', 'losing_pitcher',
                                    'save', 'time', 'd_or_n', 'attendance', 'streak', 'double_header', 'runs_per_game',
                                    'runs_moving_avg', 'runs_allowed_moving_avg', 'win_pct', 'opening_day'])
   # for team in teams:
    for year in range (2000, 2018):
        try:
            df = pd.DataFrame(columns=['date', 'box', 'team', 'at', 'opponent', 'w_or_l', 'runs', 'runs_allowed',
                                       'innings', 'WL_record', 'rank', 'gb', 'winning_pitcher', 'losing_pitcher',
                                       'save', 'time', 'd_or_n', 'attendance', 'streak'])
            urls = requests.get('http://www.baseball-reference.com/teams/' + 'COL' + '/' + str(year) + '-schedule-scores.shtml')
            soup = BeautifulSoup(urls.text, 'html.parser')
            for game in soup.find('table', {'class':'stats_table'}).find_all('tr'):
                results = []
                for element in game.find_all('td'):
                    results.append(element.text)
                    if len(results) == 19:
                        df.loc[len(df)] = results



            # double header variable for when 2 teams play same day in front of same crowd
            double_header = '('
            df['double_header'] = [1 if double_header in x else 0 for x in df.date]

            # convert date column to datetime
            df.date = df.date.str.replace(r"\(.*\)","")
            date = []
            for day in df.date:
                split = day.split(', ')
                date.append(split[1] + ', ' + str(year))
            df.date = date
            df.date = pd.to_datetime(df.date)

            # shift WL_record, rank, gb, streak and fill in the first value
            df.WL_record = df.WL_record.shift()
            df.WL_record.fillna('0-0', inplace=True)

            for col in ['rank', 'gb', 'streak']:
                df[col] = df[col].shift()
                df[col].fillna('0', inplace=True)

            # rolling means for runs and runs allowed
            df['runs_per_game'] = [0 if x == min(df.date) else df[df.date < x].runs.astype(int).mean() for x in df.date]
            df['runs_moving_avg'] = df.runs.rolling(5).mean().shift()
            df['runs_allowed_moving_avg'] = df.runs_allowed.rolling(5).mean().shift()
            df.runs_allowed_moving_avg.fillna(df.iloc[5].runs_allowed_moving_avg, inplace=True)

            # win_pct of last ten games
            win_pct = [list(df.w_or_l[i-10:i]).count('W') / 10. for i in range(10, len(df))]
            for i in range(10):
                win_pct.insert(0, None)
            df['win_pct'] = win_pct

            # return home games only
            df = df[~df['at'].str.contains('@')].reset_index(drop=True)

            # make dummny variable if game was on opening day or not
            df['opening_day'] = [1 if x == min(df.date) else 0 for x in df.date]

            # concatenate the two dataframes 
            team_df = pd.concat([team_df, df]).reset_index(drop=True)

            #sleep
            sleep(0.5)

        except:
            pass

    # save it to csv
    team_df.to_csv('/Users/andreewerner/Desktop/mlb-game-data-COL.csv', index=False, encoding='utf-8')
    return team_df
games()

,date,box,team,at,opponent,w_or_l,runs,runs_allowed,innings,WL_record,rank,gb,winning_pitcher,losing_pitcher,save,time,d_or_n,attendance,streak,double_header,runs_per_game,runs_moving_avg,runs_allowed_moving_avg,win_pct,opening_day
0,2000-04-10,boxscore,COL,,CIN,W,7,5,,2-4,5,3.0,Arrojo,Parris,Belinda,3:00,D,"48,094",-,0,4.000000,4.8,5.0,NaN,1
1,2000-04-11,boxscore,COL,,CIN,L,3,10,,3-4,3,3.0,Villone,Bohanon,,2:54,N,"40,488",+,0,4.428571,5.2,5.4,NaN,0
2,2000-04-12,boxscore,COL,,CIN,W,7,5,,3-5,4,3.0,Jimenez,Reyes,Tavarez,2:50,D,"38,339",-,0,4.250000,4.6,5.6,NaN,0
3,2000-04-13,boxscore,COL,,STL,W,12,6,,4-5,4,2.0,Croushore,Kile,,3:08,N,"39,365",+,0,4.555556,5.4,5.8,NaN,0
4,2000-04-14,boxscore,COL,,STL,W,6,2,,5-5,3,2.0,Astacio,Ankiel,,2:52,N,"40,248",++,0,5.300000,7.0,6.6,0.5,0
5,2000-04-16,boxscore,COL,,STL,L,3,9,,6-5,3,2.0,Hentgen,Arrojo,,2:53,D,"43,029",+++,1,5.363636,7.0,5.6,0.6,0
6,2000-04-16,boxscore,COL,,STL,W,14,13,,6-6,3,2.5,Jimenez,Mohler,White,4:19,N,"35,095",-,1,5.363636,6.2,6.4,0.5,0
7,2000-04-28,boxscore,COL,,NYM,W,12,5,,10-12,4,4.5,Astacio,Hampton,,3:00,N,"45,366",---,0,5.227273,4.4,7.0,0.4,0
8,2000-04-29,boxscore,COL,,NYM,L,6,13,,11-12,3,3.0,Reed,Yoshii,,2:56,D,"41,220",+,0,5.521739,5.6,7.2,0.4,0
9,2000-04-30,boxscore,COL,,NYM,L,11,14,,11-13,3,4.0,Leiter,Bohanon,,3:15,D,"36,252",-,0,5.541667,5.4,8.6,0.3,0


In [113]:
def games():
    team_df = pd.DataFrame(columns=['date', 'box', 'team', 'at', 'opponent', 'w_or_l', 'runs', 'runs_allowed',
                                    'innings', 'WL_record', 'rank', 'gb', 'winning_pitcher', 'losing_pitcher',
                                    'save', 'time', 'd_or_n', 'attendance', 'streak', 'double_header', 'runs_per_game',
                                    'runs_moving_avg', 'runs_allowed_moving_avg', 'win_pct', 'opening_day'])
   # for team in teams:
    for year in range (2000, 2018):
        try:
            df = pd.DataFrame(columns=['date', 'box', 'team', 'at', 'opponent', 'w_or_l', 'runs', 'runs_allowed',
                                       'innings', 'WL_record', 'rank', 'gb', 'winning_pitcher', 'losing_pitcher',
                                       'save', 'time', 'd_or_n', 'attendance', 'streak'])
            urls = requests.get('http://www.baseball-reference.com/teams/' + 'DET' + '/' + str(year) + '-schedule-scores.shtml')
            soup = BeautifulSoup(urls.text, 'html.parser')
            for game in soup.find('table', {'class':'stats_table'}).find_all('tr'):
                results = []
                for element in game.find_all('td'):
                    results.append(element.text)
                    if len(results) == 19:
                        df.loc[len(df)] = results



            # double header variable for when 2 teams play same day in front of same crowd
            double_header = '('
            df['double_header'] = [1 if double_header in x else 0 for x in df.date]

            # convert date column to datetime
            df.date = df.date.str.replace(r"\(.*\)","")
            date = []
            for day in df.date:
                split = day.split(', ')
                date.append(split[1] + ', ' + str(year))
            df.date = date
            df.date = pd.to_datetime(df.date)

            # shift WL_record, rank, gb, streak and fill in the first value
            df.WL_record = df.WL_record.shift()
            df.WL_record.fillna('0-0', inplace=True)

            for col in ['rank', 'gb', 'streak']:
                df[col] = df[col].shift()
                df[col].fillna('0', inplace=True)

            # rolling means for runs and runs allowed
            df['runs_per_game'] = [0 if x == min(df.date) else df[df.date < x].runs.astype(int).mean() for x in df.date]
            df['runs_moving_avg'] = df.runs.rolling(5).mean().shift()
            df['runs_allowed_moving_avg'] = df.runs_allowed.rolling(5).mean().shift()
            df.runs_allowed_moving_avg.fillna(df.iloc[5].runs_allowed_moving_avg, inplace=True)

            # win_pct of last ten games
            win_pct = [list(df.w_or_l[i-10:i]).count('W') / 10. for i in range(10, len(df))]
            for i in range(10):
                win_pct.insert(0, None)
            df['win_pct'] = win_pct

            # return home games only
            df = df[~df['at'].str.contains('@')].reset_index(drop=True)

            # make dummny variable if game was on opening day or not
            df['opening_day'] = [1 if x == min(df.date) else 0 for x in df.date]

            # concatenate the two dataframes 
            team_df = pd.concat([team_df, df]).reset_index(drop=True)

            #sleep
            sleep(0.5)

        except:
            pass

    # save it to csv
    team_df.to_csv('/Users/andreewerner/Desktop/mlb-game-data-DET.csv', index=False, encoding='utf-8')
    return team_df
games()

,date,box,team,at,opponent,w_or_l,runs,runs_allowed,innings,WL_record,rank,gb,winning_pitcher,losing_pitcher,save,time,d_or_n,attendance,streak,double_header,runs_per_game,runs_moving_avg,runs_allowed_moving_avg,win_pct,opening_day
0,2000-04-11,boxscore,DET,,SEA,W,5,2,,1-5,5,3.0,Moehler,Garcia,Jones,3:12,D,"39,168",-----,0,4.500000,4.0,7.6,NaN,1
1,2000-04-12,boxscore,DET,,SEA,L,0,4,,2-5,4,3.5,Sele,Mlicki,,2:24,N,"21,405",+,0,4.571429,4.8,7.4,NaN,0
2,2000-04-13,boxscore,DET,,SEA,W,2,0,,2-6,4,4.5,Brocail,Rhodes,Jones,2:58,N,"20,781",-,0,4.000000,4.4,6.6,NaN,0
3,2000-04-14,boxscore,DET,,TBD,W,10,5,,3-6,4,4.0,Nitkowski,Rupe,,2:47,N,"25,043",+,0,3.777778,2.8,3.8,NaN,0
4,2000-04-15,boxscore,DET,,TBD,L,0,7,,4-6,4,3.0,Gooden,Weaver,,2:38,N,"25,969",++,0,4.400000,4.6,4.4,0.4,0
5,2000-04-16,boxscore,DET,,TBD,L,6,7,,4-7,4,3.0,Hernandez,Jones,,3:01,D,"24,686",-,0,4.000000,3.4,3.6,0.3,0
6,2000-04-18,boxscore,DET,,BOS,L,0,7,,4-8,4,4.0,Schourek,Mlicki,Lowe,2:34,N,"21,632",--,0,4.166667,3.6,4.6,0.3,0
7,2000-04-19,boxscore,DET,,BOS,L,0,10,,4-9,4,4.0,Rose,Nomo,,3:00,N,"20,526",---,0,3.846154,3.6,5.2,0.3,0
8,2000-04-28,boxscore,DET,,CHW,L,2,3,,5-15,5,8.5,Baldwin,Weaver,Foulke,2:24,N,"26,916",-,0,3.550000,3.8,8.2,0.1,0
9,2000-04-29,boxscore,DET,,CHW,L,1,2,,5-16,5,10.0,Eldred,Mlicki,Foulke,2:37,D,"28,393",--,0,3.476190,3.0,6.0,0.1,0


In [114]:
def games():
    team_df = pd.DataFrame(columns=['date', 'box', 'team', 'at', 'opponent', 'w_or_l', 'runs', 'runs_allowed',
                                    'innings', 'WL_record', 'rank', 'gb', 'winning_pitcher', 'losing_pitcher',
                                    'save', 'time', 'd_or_n', 'attendance', 'streak', 'double_header', 'runs_per_game',
                                    'runs_moving_avg', 'runs_allowed_moving_avg', 'win_pct', 'opening_day'])
   # for team in teams:
    for year in range (2000, 2018):
        try:
            df = pd.DataFrame(columns=['date', 'box', 'team', 'at', 'opponent', 'w_or_l', 'runs', 'runs_allowed',
                                       'innings', 'WL_record', 'rank', 'gb', 'winning_pitcher', 'losing_pitcher',
                                       'save', 'time', 'd_or_n', 'attendance', 'streak'])
            urls = requests.get('http://www.baseball-reference.com/teams/' + 'FLA' + '/' + str(year) + '-schedule-scores.shtml')
            soup = BeautifulSoup(urls.text, 'html.parser')
            for game in soup.find('table', {'class':'stats_table'}).find_all('tr'):
                results = []
                for element in game.find_all('td'):
                    results.append(element.text)
                    if len(results) == 19:
                        df.loc[len(df)] = results



            # double header variable for when 2 teams play same day in front of same crowd
            double_header = '('
            df['double_header'] = [1 if double_header in x else 0 for x in df.date]

            # convert date column to datetime
            df.date = df.date.str.replace(r"\(.*\)","")
            date = []
            for day in df.date:
                split = day.split(', ')
                date.append(split[1] + ', ' + str(year))
            df.date = date
            df.date = pd.to_datetime(df.date)

            # shift WL_record, rank, gb, streak and fill in the first value
            df.WL_record = df.WL_record.shift()
            df.WL_record.fillna('0-0', inplace=True)

            for col in ['rank', 'gb', 'streak']:
                df[col] = df[col].shift()
                df[col].fillna('0', inplace=True)

            # rolling means for runs and runs allowed
            df['runs_per_game'] = [0 if x == min(df.date) else df[df.date < x].runs.astype(int).mean() for x in df.date]
            df['runs_moving_avg'] = df.runs.rolling(5).mean().shift()
            df['runs_allowed_moving_avg'] = df.runs_allowed.rolling(5).mean().shift()
            df.runs_allowed_moving_avg.fillna(df.iloc[5].runs_allowed_moving_avg, inplace=True)

            # win_pct of last ten games
            win_pct = [list(df.w_or_l[i-10:i]).count('W') / 10. for i in range(10, len(df))]
            for i in range(10):
                win_pct.insert(0, None)
            df['win_pct'] = win_pct

            # return home games only
            df = df[~df['at'].str.contains('@')].reset_index(drop=True)

            # make dummny variable if game was on opening day or not
            df['opening_day'] = [1 if x == min(df.date) else 0 for x in df.date]

            # concatenate the two dataframes 
            team_df = pd.concat([team_df, df]).reset_index(drop=True)

            #sleep
            sleep(0.5)

        except:
            pass

    # save it to csv
    team_df.to_csv('/Users/andreewerner/Desktop/mlb-game-data-FLA.csv', index=False, encoding='utf-8')
    return team_df
games()

,date,box,team,at,opponent,w_or_l,runs,runs_allowed,innings,WL_record,rank,gb,winning_pitcher,losing_pitcher,save,time,d_or_n,attendance,streak,double_header,runs_per_game,runs_moving_avg,runs_allowed_moving_avg,win_pct,opening_day
0,2000-04-03,boxscore,FLA,,SFG,W,6,4,,0-0,0,0,Fernandez,Hernandez,Alfonseca,2:46,N,"35,101",0,0,0.000000,NaN,5.0,NaN,1
1,2000-04-04,boxscore,FLA,,SFG,L,0,3,,1-0,1,Tied,Ortiz,Dempster,Nen,2:59,N,"9,524",+,0,6.000000,NaN,5.0,NaN,0
2,2000-04-05,boxscore,FLA,,SFG,L,9,11,,1-1,2,0.5,Johnstone,Alfonseca,Nen,3:42,N,"8,365",-,0,3.000000,NaN,5.0,NaN,0
3,2000-04-06,boxscore,FLA,,SFG,W-wo,5,4,,1-2,3,1.0,Miceli,Johnstone,,3:29,D,"7,741",--,0,5.000000,NaN,5.0,NaN,0
4,2000-04-07,boxscore,FLA,,COL,W,4,3,,2-2,2,0.5,Penny,Yoshii,Alfonseca,2:38,N,"13,803",+,0,5.000000,NaN,5.0,NaN,0
5,2000-04-08,boxscore,FLA,,COL,L,2,4,,3-2,1,up 0.5,Jimenez,Fernandez,Lee,2:51,N,"20,077",++,0,4.800000,4.8,5.0,NaN,0
6,2000-04-09,boxscore,FLA,,COL,W,7,6,,3-3,2,0.5,Dempster,Astacio,Alfonseca,3:10,D,"16,463",-,0,4.333333,4.0,5.0,NaN,0
7,2000-04-18,boxscore,FLA,,PIT,W,12,5,,8-6,1,up 1.0,Penny,Cordova,,2:59,N,"9,494",++,0,5.000000,4.6,4.4,0.6,0
8,2000-04-19,boxscore,FLA,,PIT,L,1,5,,9-6,1,up 1.0,Ritchie,Fernandez,,2:57,N,"10,911",+++,0,5.466667,7.0,4.6,0.6,0
9,2000-04-20,boxscore,FLA,,PIT,W-wo,3,2,14,9-7,1,Tied,Darensbourg,Williams,,4:31,N,"9,396",-,0,5.187500,5.4,4.8,0.6,0


In [115]:
def games():
    team_df = pd.DataFrame(columns=['date', 'box', 'team', 'at', 'opponent', 'w_or_l', 'runs', 'runs_allowed',
                                    'innings', 'WL_record', 'rank', 'gb', 'winning_pitcher', 'losing_pitcher',
                                    'save', 'time', 'd_or_n', 'attendance', 'streak', 'double_header', 'runs_per_game',
                                    'runs_moving_avg', 'runs_allowed_moving_avg', 'win_pct', 'opening_day'])
   # for team in teams:
    for year in range (2000, 2018):
        try:
            df = pd.DataFrame(columns=['date', 'box', 'team', 'at', 'opponent', 'w_or_l', 'runs', 'runs_allowed',
                                       'innings', 'WL_record', 'rank', 'gb', 'winning_pitcher', 'losing_pitcher',
                                       'save', 'time', 'd_or_n', 'attendance', 'streak'])
            urls = requests.get('http://www.baseball-reference.com/teams/' + 'HOU' + '/' + str(year) + '-schedule-scores.shtml')
            soup = BeautifulSoup(urls.text, 'html.parser')
            for game in soup.find('table', {'class':'stats_table'}).find_all('tr'):
                results = []
                for element in game.find_all('td'):
                    results.append(element.text)
                    if len(results) == 19:
                        df.loc[len(df)] = results



            # double header variable for when 2 teams play same day in front of same crowd
            double_header = '('
            df['double_header'] = [1 if double_header in x else 0 for x in df.date]

            # convert date column to datetime
            df.date = df.date.str.replace(r"\(.*\)","")
            date = []
            for day in df.date:
                split = day.split(', ')
                date.append(split[1] + ', ' + str(year))
            df.date = date
            df.date = pd.to_datetime(df.date)

            # shift WL_record, rank, gb, streak and fill in the first value
            df.WL_record = df.WL_record.shift()
            df.WL_record.fillna('0-0', inplace=True)

            for col in ['rank', 'gb', 'streak']:
                df[col] = df[col].shift()
                df[col].fillna('0', inplace=True)

            # rolling means for runs and runs allowed
            df['runs_per_game'] = [0 if x == min(df.date) else df[df.date < x].runs.astype(int).mean() for x in df.date]
            df['runs_moving_avg'] = df.runs.rolling(5).mean().shift()
            df['runs_allowed_moving_avg'] = df.runs_allowed.rolling(5).mean().shift()
            df.runs_allowed_moving_avg.fillna(df.iloc[5].runs_allowed_moving_avg, inplace=True)

            # win_pct of last ten games
            win_pct = [list(df.w_or_l[i-10:i]).count('W') / 10. for i in range(10, len(df))]
            for i in range(10):
                win_pct.insert(0, None)
            df['win_pct'] = win_pct

            # return home games only
            df = df[~df['at'].str.contains('@')].reset_index(drop=True)

            # make dummny variable if game was on opening day or not
            df['opening_day'] = [1 if x == min(df.date) else 0 for x in df.date]

            # concatenate the two dataframes 
            team_df = pd.concat([team_df, df]).reset_index(drop=True)

            #sleep
            sleep(0.5)

        except:
            pass

    # save it to csv
    team_df.to_csv('/Users/andreewerner/Desktop/mlb-game-data-HOU.csv', index=False, encoding='utf-8')
    return team_df
games()

,date,box,team,at,opponent,w_or_l,runs,runs_allowed,innings,WL_record,rank,gb,winning_pitcher,losing_pitcher,save,time,d_or_n,attendance,streak,double_header,runs_per_game,runs_moving_avg,runs_allowed_moving_avg,win_pct,opening_day
0,2000-04-07,boxscore,HOU,,PHI,L,1,4,,2-1,2,1.0,Wolf,Dotel,Gomes,2:43,N,"41,583",-,0,5.666667,NaN,4.6,NaN,1
1,2000-04-08,boxscore,HOU,,PHI,W,8,5,,2-2,3,1.0,Maddux,Brock,Wagner,3:06,D,"39,018",--,0,4.500000,NaN,4.6,NaN,0
2,2000-04-09,boxscore,HOU,,PHI,L,2,3,,3-2,2,1.0,Schrenk,Powell,Gomes,3:22,D,"38,784",+,0,5.200000,5.2,4.6,NaN,0
3,2000-04-10,boxscore,HOU,,STL,L,7,8,,3-3,2,2.0,Hentgen,Lima,Veres,3:03,N,"36,324",-,0,4.666667,4.6,4.8,NaN,0
4,2000-04-11,boxscore,HOU,,STL,L,6,10,,3-4,3,3.0,Stephenson,Holt,,2:55,N,"37,463",--,0,5.000000,3.8,6.0,NaN,0
5,2000-04-12,boxscore,HOU,,STL,W,7,5,,3-5,4,4.0,Perez,Wainhouse,Wagner,2:51,N,"38,783",---,0,5.125000,4.8,6.0,NaN,0
6,2000-04-21,boxscore,HOU,,SDP,L,2,7,,6-8,3,3.5,Williams,Lima,,2:51,N,"42,092",+,0,5.500000,5.8,6.0,0.4,0
7,2000-04-22,boxscore,HOU,,SDP,L,6,8,10,6-9,3,4.0,Cunnane,Perez,Hoffman,3:29,D,"42,121",-,0,5.266667,4.2,6.6,0.3,0
8,2000-04-23,boxscore,HOU,,SDP,L,10,11,,6-10,4,4.0,Whisenant,Wagner,Hoffman,3:17,D,"38,695",--,0,5.312500,4.8,7.2,0.3,0
9,2000-04-25,boxscore,HOU,,CHC,W,11,7,,6-11,4,5.0,Reynolds,Downs,,3:34,N,"37,939",---,0,5.588235,6.2,6.8,0.3,0


In [116]:
def games():
    team_df = pd.DataFrame(columns=['date', 'box', 'team', 'at', 'opponent', 'w_or_l', 'runs', 'runs_allowed',
                                    'innings', 'WL_record', 'rank', 'gb', 'winning_pitcher', 'losing_pitcher',
                                    'save', 'time', 'd_or_n', 'attendance', 'streak', 'double_header', 'runs_per_game',
                                    'runs_moving_avg', 'runs_allowed_moving_avg', 'win_pct', 'opening_day'])
   # for team in teams:
    for year in range (2000, 2018):
        try:
            df = pd.DataFrame(columns=['date', 'box', 'team', 'at', 'opponent', 'w_or_l', 'runs', 'runs_allowed',
                                       'innings', 'WL_record', 'rank', 'gb', 'winning_pitcher', 'losing_pitcher',
                                       'save', 'time', 'd_or_n', 'attendance', 'streak'])
            urls = requests.get('http://www.baseball-reference.com/teams/' + 'KCR' + '/' + str(year) + '-schedule-scores.shtml')
            soup = BeautifulSoup(urls.text, 'html.parser')
            for game in soup.find('table', {'class':'stats_table'}).find_all('tr'):
                results = []
                for element in game.find_all('td'):
                    results.append(element.text)
                    if len(results) == 19:
                        df.loc[len(df)] = results



            # double header variable for when 2 teams play same day in front of same crowd
            double_header = '('
            df['double_header'] = [1 if double_header in x else 0 for x in df.date]

            # convert date column to datetime
            df.date = df.date.str.replace(r"\(.*\)","")
            date = []
            for day in df.date:
                split = day.split(', ')
                date.append(split[1] + ', ' + str(year))
            df.date = date
            df.date = pd.to_datetime(df.date)

            # shift WL_record, rank, gb, streak and fill in the first value
            df.WL_record = df.WL_record.shift()
            df.WL_record.fillna('0-0', inplace=True)

            for col in ['rank', 'gb', 'streak']:
                df[col] = df[col].shift()
                df[col].fillna('0', inplace=True)

            # rolling means for runs and runs allowed
            df['runs_per_game'] = [0 if x == min(df.date) else df[df.date < x].runs.astype(int).mean() for x in df.date]
            df['runs_moving_avg'] = df.runs.rolling(5).mean().shift()
            df['runs_allowed_moving_avg'] = df.runs_allowed.rolling(5).mean().shift()
            df.runs_allowed_moving_avg.fillna(df.iloc[5].runs_allowed_moving_avg, inplace=True)

            # win_pct of last ten games
            win_pct = [list(df.w_or_l[i-10:i]).count('W') / 10. for i in range(10, len(df))]
            for i in range(10):
                win_pct.insert(0, None)
            df['win_pct'] = win_pct

            # return home games only
            df = df[~df['at'].str.contains('@')].reset_index(drop=True)

            # make dummny variable if game was on opening day or not
            df['opening_day'] = [1 if x == min(df.date) else 0 for x in df.date]

            # concatenate the two dataframes 
            team_df = pd.concat([team_df, df]).reset_index(drop=True)

            #sleep
            sleep(0.5)

        except:
            pass

    # save it to csv
    team_df.to_csv('/Users/andreewerner/Desktop/mlb-game-data-KCR.csv', index=False, encoding='utf-8')
    return team_df
games()

,date,box,team,at,opponent,w_or_l,runs,runs_allowed,innings,WL_record,rank,gb,winning_pitcher,losing_pitcher,save,time,d_or_n,attendance,streak,double_header,runs_per_game,runs_moving_avg,runs_allowed_moving_avg,win_pct,opening_day
0,2000-04-07,boxscore,KCR,,MIN,W,10,6,,2-2,1,Tied,Fussell,Miller,,3:11,D,"40,474",++,0,5.000000,NaN,4.6,NaN,1
1,2000-04-08,boxscore,KCR,,MIN,W,5,2,,3-2,1,Tied,Suppan,Radke,Santiago,2:23,D,"17,133",+++,0,6.000000,6.0,4.6,NaN,0
2,2000-04-09,boxscore,KCR,,MIN,L,7,13,,4-2,1,Tied,Milton,Witasick,,2:57,D,"20,480",++++,0,5.833333,6.2,4.0,NaN,0
3,2000-04-10,boxscore,KCR,,MIN,W-wo,6,5,,4-3,2,0.5,Bottalico,Hawkins,,2:54,D,"13,252",-,0,6.000000,7.0,5.4,NaN,0
4,2000-04-11,boxscore,KCR,,BAL,W-wo,7,5,12,5-3,2,0.5,Santiago,Worrell,,3:52,N,"13,080",+,0,6.000000,7.4,5.8,NaN,0
5,2000-04-12,boxscore,KCR,,BAL,W-wo,7,6,,6-3,2,0.5,Santiago,Trombley,,3:01,N,"15,228",++,0,6.111111,7.0,6.2,NaN,0
6,2000-04-13,boxscore,KCR,,BAL,W-wo,6,5,,7-3,2,0.5,Bottalico,Ryan,,2:24,D,"23,724",+++,0,6.200000,6.4,6.2,0.4,0
7,2000-04-25,boxscore,KCR,,TBD,W-wo,7,6,,8-12,4,5.0,Reichert,Lopez,,3:19,N,"14,677",---------,0,5.050000,4.4,7.6,0.0,0
8,2000-04-26,boxscore,KCR,,TBD,W-wo,7,6,,9-12,4,4.5,Bottalico,Lopez,,3:22,N,"14,209",+,0,5.142857,4.6,7.4,0.0,0
9,2000-04-28,boxscore,KCR,,SEA,W,8,5,,10-12,4,4.5,Batista,Tomko,Bottalico,3:24,N,"36,317",++,0,5.227273,4.6,6.8,0.0,0


In [117]:
def games():
    team_df = pd.DataFrame(columns=['date', 'box', 'team', 'at', 'opponent', 'w_or_l', 'runs', 'runs_allowed',
                                    'innings', 'WL_record', 'rank', 'gb', 'winning_pitcher', 'losing_pitcher',
                                    'save', 'time', 'd_or_n', 'attendance', 'streak', 'double_header', 'runs_per_game',
                                    'runs_moving_avg', 'runs_allowed_moving_avg', 'win_pct', 'opening_day'])
   # for team in teams:
    for year in range (2000, 2018):
        try:
            df = pd.DataFrame(columns=['date', 'box', 'team', 'at', 'opponent', 'w_or_l', 'runs', 'runs_allowed',
                                       'innings', 'WL_record', 'rank', 'gb', 'winning_pitcher', 'losing_pitcher',
                                       'save', 'time', 'd_or_n', 'attendance', 'streak'])
            urls = requests.get('http://www.baseball-reference.com/teams/' + 'LAA' + '/' + str(year) + '-schedule-scores.shtml')
            soup = BeautifulSoup(urls.text, 'html.parser')
            for game in soup.find('table', {'class':'stats_table'}).find_all('tr'):
                results = []
                for element in game.find_all('td'):
                    results.append(element.text)
                    if len(results) == 19:
                        df.loc[len(df)] = results



            # double header variable for when 2 teams play same day in front of same crowd
            double_header = '('
            df['double_header'] = [1 if double_header in x else 0 for x in df.date]

            # convert date column to datetime
            df.date = df.date.str.replace(r"\(.*\)","")
            date = []
            for day in df.date:
                split = day.split(', ')
                date.append(split[1] + ', ' + str(year))
            df.date = date
            df.date = pd.to_datetime(df.date)

            # shift WL_record, rank, gb, streak and fill in the first value
            df.WL_record = df.WL_record.shift()
            df.WL_record.fillna('0-0', inplace=True)

            for col in ['rank', 'gb', 'streak']:
                df[col] = df[col].shift()
                df[col].fillna('0', inplace=True)

            # rolling means for runs and runs allowed
            df['runs_per_game'] = [0 if x == min(df.date) else df[df.date < x].runs.astype(int).mean() for x in df.date]
            df['runs_moving_avg'] = df.runs.rolling(5).mean().shift()
            df['runs_allowed_moving_avg'] = df.runs_allowed.rolling(5).mean().shift()
            df.runs_allowed_moving_avg.fillna(df.iloc[5].runs_allowed_moving_avg, inplace=True)

            # win_pct of last ten games
            win_pct = [list(df.w_or_l[i-10:i]).count('W') / 10. for i in range(10, len(df))]
            for i in range(10):
                win_pct.insert(0, None)
            df['win_pct'] = win_pct

            # return home games only
            df = df[~df['at'].str.contains('@')].reset_index(drop=True)

            # make dummny variable if game was on opening day or not
            df['opening_day'] = [1 if x == min(df.date) else 0 for x in df.date]

            # concatenate the two dataframes 
            team_df = pd.concat([team_df, df]).reset_index(drop=True)

            #sleep
            sleep(0.5)

        except:
            pass

    # save it to csv
    team_df.to_csv('/Users/andreewerner/Desktop/mlb-game-data-LAA.csv', index=False, encoding='utf-8')
    return team_df
games()

,date,box,team,at,opponent,w_or_l,runs,runs_allowed,innings,WL_record,rank,gb,winning_pitcher,losing_pitcher,save,time,d_or_n,attendance,streak,double_header,runs_per_game,runs_moving_avg,runs_allowed_moving_avg,win_pct,opening_day
0,2005-04-05,boxscore,LAA,,TEX,W,3,2,,0-0,0,0,Colon,Drese,Rodriguez,2:27,N,"43,590",0,0,0.000000,NaN,4.0,NaN,1
1,2005-04-06,boxscore,LAA,,TEX,L,2,3,12,1-0,1,up 0.5,Shouse,Prinz,,3:50,N,"35,077",+,0,3.000000,NaN,4.0,NaN,0
2,2005-04-07,boxscore,LAA,,TEX,W-wo,7,6,,1-1,1,Tied,Shields,Shouse,,3:15,N,"35,169",-,0,2.500000,NaN,4.0,NaN,0
3,2005-04-08,boxscore,LAA,,KCR,L,2,6,,2-1,1,Tied,Bautista,Byrd,,2:21,N,"39,167",+,0,4.000000,NaN,4.0,NaN,0
4,2005-04-09,boxscore,LAA,,KCR,W,8,3,,2-2,1,Tied,Gregg,Lima,,2:45,N,"42,007",-,0,3.500000,NaN,4.0,NaN,0
5,2005-04-10,boxscore,LAA,,KCR,L,3,8,,3-2,1,up 1.0,Anderson,Colon,,3:05,D,"40,689",+,0,4.400000,4.4,4.0,NaN,0
6,2005-04-18,boxscore,LAA,,SEA,W,6,1,,6-6,1,Tied,Byrd,Sele,,2:36,N,"39,638",--,0,5.166667,6.0,4.8,0.4,0
7,2005-04-19,boxscore,LAA,,SEA,L,3,5,,7-6,1,Tied,Moyer,Gregg,Guardado,3:11,D,"38,667",+,0,5.230769,4.6,3.4,0.5,0
8,2005-04-20,boxscore,LAA,,CLE,W,2,0,,7-7,1,Tied,Colon,Westbrook,Rodriguez,2:23,N,"42,531",-,0,5.071429,4.2,3.0,0.5,0
9,2005-04-21,boxscore,LAA,,CLE,W-wo,6,5,10,8-7,1,Tied,Rodriguez,Davis,,3:35,N,"39,673",+,0,4.866667,3.4,2.8,0.5,0


In [118]:
def games():
    team_df = pd.DataFrame(columns=['date', 'box', 'team', 'at', 'opponent', 'w_or_l', 'runs', 'runs_allowed',
                                    'innings', 'WL_record', 'rank', 'gb', 'winning_pitcher', 'losing_pitcher',
                                    'save', 'time', 'd_or_n', 'attendance', 'streak', 'double_header', 'runs_per_game',
                                    'runs_moving_avg', 'runs_allowed_moving_avg', 'win_pct', 'opening_day'])
   # for team in teams:
    for year in range (2000, 2018):
        try:
            df = pd.DataFrame(columns=['date', 'box', 'team', 'at', 'opponent', 'w_or_l', 'runs', 'runs_allowed',
                                       'innings', 'WL_record', 'rank', 'gb', 'winning_pitcher', 'losing_pitcher',
                                       'save', 'time', 'd_or_n', 'attendance', 'streak'])
            urls = requests.get('http://www.baseball-reference.com/teams/' + 'LAD' + '/' + str(year) + '-schedule-scores.shtml')
            soup = BeautifulSoup(urls.text, 'html.parser')
            for game in soup.find('table', {'class':'stats_table'}).find_all('tr'):
                results = []
                for element in game.find_all('td'):
                    results.append(element.text)
                    if len(results) == 19:
                        df.loc[len(df)] = results



            # double header variable for when 2 teams play same day in front of same crowd
            double_header = '('
            df['double_header'] = [1 if double_header in x else 0 for x in df.date]

            # convert date column to datetime
            df.date = df.date.str.replace(r"\(.*\)","")
            date = []
            for day in df.date:
                split = day.split(', ')
                date.append(split[1] + ', ' + str(year))
            df.date = date
            df.date = pd.to_datetime(df.date)

            # shift WL_record, rank, gb, streak and fill in the first value
            df.WL_record = df.WL_record.shift()
            df.WL_record.fillna('0-0', inplace=True)

            for col in ['rank', 'gb', 'streak']:
                df[col] = df[col].shift()
                df[col].fillna('0', inplace=True)

            # rolling means for runs and runs allowed
            df['runs_per_game'] = [0 if x == min(df.date) else df[df.date < x].runs.astype(int).mean() for x in df.date]
            df['runs_moving_avg'] = df.runs.rolling(5).mean().shift()
            df['runs_allowed_moving_avg'] = df.runs_allowed.rolling(5).mean().shift()
            df.runs_allowed_moving_avg.fillna(df.iloc[5].runs_allowed_moving_avg, inplace=True)

            # win_pct of last ten games
            win_pct = [list(df.w_or_l[i-10:i]).count('W') / 10. for i in range(10, len(df))]
            for i in range(10):
                win_pct.insert(0, None)
            df['win_pct'] = win_pct

            # return home games only
            df = df[~df['at'].str.contains('@')].reset_index(drop=True)

            # make dummny variable if game was on opening day or not
            df['opening_day'] = [1 if x == min(df.date) else 0 for x in df.date]

            # concatenate the two dataframes 
            team_df = pd.concat([team_df, df]).reset_index(drop=True)

            #sleep
            sleep(0.5)

        except:
            pass

    # save it to csv
    team_df.to_csv('/Users/andreewerner/Desktop/mlb-game-data-LAD.csv', index=False, encoding='utf-8')
    return team_df
games()

,date,box,team,at,opponent,w_or_l,runs,runs_allowed,innings,WL_record,rank,gb,winning_pitcher,losing_pitcher,save,time,d_or_n,attendance,streak,double_header,runs_per_game,runs_moving_avg,runs_allowed_moving_avg,win_pct,opening_day
0,2000-04-14,boxscore,LAD,,CIN,W,8,1,,6-3,2,0.5,Hershiser,Bell,,2:45,D,"53,223",++++,0,6.444444,6.0,4.8,NaN,1
1,2000-04-15,boxscore,LAD,,CIN,L,4,5,,7-3,2,0.5,Williamson,Mills,Graves,3:30,N,"51,179",+++++,0,6.600000,7.4,4.6,0.7,0
2,2000-04-16,boxscore,LAD,,CIN,L,3,5,,7-4,2,1.5,Parris,Park,Graves,3:18,D,"52,594",-,0,6.363636,7.0,4.6,0.6,0
3,2000-04-18,boxscore,LAD,,HOU,W,5,3,,7-5,2,2.0,Dreifort,Dotel,Shaw,2:58,N,"26,298",--,0,6.083333,6.4,4.6,0.5,0
4,2000-04-19,boxscore,LAD,,HOU,L,3,10,,8-5,2,1.5,Reynolds,Hershiser,,3:32,N,"29,717",+,0,6.000000,6.2,4.2,0.6,0
5,2000-04-28,boxscore,LAD,,FLA,W,5,3,,11-10,2,2.5,Adams,Penny,Shaw,2:45,N,"46,012",----,0,5.761905,3.0,3.2,0.4,0
6,2000-04-29,boxscore,LAD,,FLA,W-wo,13,12,,12-10,2,1.5,Shaw,Alfonseca,,3:49,N,"35,583",+,0,5.727273,1.8,3.2,0.5,0
7,2000-04-30,boxscore,LAD,,FLA,W,7,1,,13-10,2,1.5,Brown,Fernandez,,2:33,D,"31,940",++,0,6.043478,4.4,5.4,0.4,0
8,2000-05-01,boxscore,LAD,,ATL,L,1,2,,14-10,2,1.5,Millwood,Gagne,Rocker,2:52,N,"28,790",+++,0,6.083333,5.8,5.4,0.5,0
9,2000-05-02,boxscore,LAD,,ATL,L,3,5,,14-11,2,2.0,Chen,Adams,Rocker,3:11,N,"30,246",-,0,5.880000,5.8,4.8,0.4,0


In [119]:
def games():
    team_df = pd.DataFrame(columns=['date', 'box', 'team', 'at', 'opponent', 'w_or_l', 'runs', 'runs_allowed',
                                    'innings', 'WL_record', 'rank', 'gb', 'winning_pitcher', 'losing_pitcher',
                                    'save', 'time', 'd_or_n', 'attendance', 'streak', 'double_header', 'runs_per_game',
                                    'runs_moving_avg', 'runs_allowed_moving_avg', 'win_pct', 'opening_day'])
   # for team in teams:
    for year in range (2000, 2018):
        try:
            df = pd.DataFrame(columns=['date', 'box', 'team', 'at', 'opponent', 'w_or_l', 'runs', 'runs_allowed',
                                       'innings', 'WL_record', 'rank', 'gb', 'winning_pitcher', 'losing_pitcher',
                                       'save', 'time', 'd_or_n', 'attendance', 'streak'])
            urls = requests.get('http://www.baseball-reference.com/teams/' + 'MIA' + '/' + str(year) + '-schedule-scores.shtml')
            soup = BeautifulSoup(urls.text, 'html.parser')
            for game in soup.find('table', {'class':'stats_table'}).find_all('tr'):
                results = []
                for element in game.find_all('td'):
                    results.append(element.text)
                    if len(results) == 19:
                        df.loc[len(df)] = results



            # double header variable for when 2 teams play same day in front of same crowd
            double_header = '('
            df['double_header'] = [1 if double_header in x else 0 for x in df.date]

            # convert date column to datetime
            df.date = df.date.str.replace(r"\(.*\)","")
            date = []
            for day in df.date:
                split = day.split(', ')
                date.append(split[1] + ', ' + str(year))
            df.date = date
            df.date = pd.to_datetime(df.date)

            # shift WL_record, rank, gb, streak and fill in the first value
            df.WL_record = df.WL_record.shift()
            df.WL_record.fillna('0-0', inplace=True)

            for col in ['rank', 'gb', 'streak']:
                df[col] = df[col].shift()
                df[col].fillna('0', inplace=True)

            # rolling means for runs and runs allowed
            df['runs_per_game'] = [0 if x == min(df.date) else df[df.date < x].runs.astype(int).mean() for x in df.date]
            df['runs_moving_avg'] = df.runs.rolling(5).mean().shift()
            df['runs_allowed_moving_avg'] = df.runs_allowed.rolling(5).mean().shift()
            df.runs_allowed_moving_avg.fillna(df.iloc[5].runs_allowed_moving_avg, inplace=True)

            # win_pct of last ten games
            win_pct = [list(df.w_or_l[i-10:i]).count('W') / 10. for i in range(10, len(df))]
            for i in range(10):
                win_pct.insert(0, None)
            df['win_pct'] = win_pct

            # return home games only
            df = df[~df['at'].str.contains('@')].reset_index(drop=True)

            # make dummny variable if game was on opening day or not
            df['opening_day'] = [1 if x == min(df.date) else 0 for x in df.date]

            # concatenate the two dataframes 
            team_df = pd.concat([team_df, df]).reset_index(drop=True)

            #sleep
            sleep(0.5)

        except:
            pass

    # save it to csv
    team_df.to_csv('/Users/andreewerner/Desktop/mlb-game-data-MIA.csv', index=False, encoding='utf-8')
    return team_df
games()

,date,box,team,at,opponent,w_or_l,runs,runs_allowed,innings,WL_record,rank,gb,winning_pitcher,losing_pitcher,save,time,d_or_n,attendance,streak,double_header,runs_per_game,runs_moving_avg,runs_allowed_moving_avg,win_pct,opening_day
0,2012-04-04,boxscore,MIA,,STL,L,1,4,,0-0,0,0,Lohse,Johnson,Motte,2:42,N,"36,601",0,0,0.000000,NaN,3.8,NaN,1
1,2012-04-13,boxscore,MIA,,HOU,W-wo,5,4,11,2-5,5,3.0,Webb,Lyon,,3:45,N,"30,169",--,0,3.142857,4.2,4.2,NaN,0
2,2012-04-14,boxscore,MIA,,HOU,L,4,5,,3-5,5,3.0,Cruz,Bell,Myers,2:48,N,"31,659",+,0,3.375000,3.6,4.4,NaN,0
3,2012-04-15,boxscore,MIA,,HOU,W-wo,5,4,11,3-6,5,4.0,Gaudin,Carpenter,,4:07,D,"34,232",-,0,3.444444,3.4,4.2,NaN,0
4,2012-04-17,boxscore,MIA,,CHC,W,5,2,,4-6,5,3.0,Cishek,Dolis,Bell,2:51,N,"24,544",+,0,3.600000,3.2,4.6,0.2,0
5,2012-04-18,boxscore,MIA,,CHC,W,9,1,,5-6,4,3.5,Buehrle,Garza,,2:38,N,"25,723",++,0,3.727273,4.0,3.6,0.3,0
6,2012-04-19,boxscore,MIA,,CHC,W,5,3,,6-6,4,3.5,Nolasco,Samardzija,Bell,3:08,D,"23,168",+++,0,4.166667,5.6,3.2,0.4,0
7,2012-04-27,boxscore,MIA,,ARI,L,0,5,,7-11,5,6.5,Saunders,Zambrano,,2:56,N,"31,949",-----,0,3.388889,1.2,3.0,0.3,0
8,2012-04-28,boxscore,MIA,,ARI,W-wo,3,2,,7-12,5,6.5,Cishek,Ziegler,,2:51,N,"33,525",------,0,3.210526,1.2,3.6,0.3,0
9,2012-04-29,boxscore,MIA,,ARI,L,4,8,,8-12,5,5.5,Miley,Johnson,,3:14,D,"34,918",+,0,3.200000,1.4,3.4,0.3,0


In [120]:
def games():
    team_df = pd.DataFrame(columns=['date', 'box', 'team', 'at', 'opponent', 'w_or_l', 'runs', 'runs_allowed',
                                    'innings', 'WL_record', 'rank', 'gb', 'winning_pitcher', 'losing_pitcher',
                                    'save', 'time', 'd_or_n', 'attendance', 'streak', 'double_header', 'runs_per_game',
                                    'runs_moving_avg', 'runs_allowed_moving_avg', 'win_pct', 'opening_day'])
   # for team in teams:
    for year in range (2000, 2018):
        try:
            df = pd.DataFrame(columns=['date', 'box', 'team', 'at', 'opponent', 'w_or_l', 'runs', 'runs_allowed',
                                       'innings', 'WL_record', 'rank', 'gb', 'winning_pitcher', 'losing_pitcher',
                                       'save', 'time', 'd_or_n', 'attendance', 'streak'])
            urls = requests.get('http://www.baseball-reference.com/teams/' + 'MIL' + '/' + str(year) + '-schedule-scores.shtml')
            soup = BeautifulSoup(urls.text, 'html.parser')
            for game in soup.find('table', {'class':'stats_table'}).find_all('tr'):
                results = []
                for element in game.find_all('td'):
                    results.append(element.text)
                    if len(results) == 19:
                        df.loc[len(df)] = results



            # double header variable for when 2 teams play same day in front of same crowd
            double_header = '('
            df['double_header'] = [1 if double_header in x else 0 for x in df.date]

            # convert date column to datetime
            df.date = df.date.str.replace(r"\(.*\)","")
            date = []
            for day in df.date:
                split = day.split(', ')
                date.append(split[1] + ', ' + str(year))
            df.date = date
            df.date = pd.to_datetime(df.date)

            # shift WL_record, rank, gb, streak and fill in the first value
            df.WL_record = df.WL_record.shift()
            df.WL_record.fillna('0-0', inplace=True)

            for col in ['rank', 'gb', 'streak']:
                df[col] = df[col].shift()
                df[col].fillna('0', inplace=True)

            # rolling means for runs and runs allowed
            df['runs_per_game'] = [0 if x == min(df.date) else df[df.date < x].runs.astype(int).mean() for x in df.date]
            df['runs_moving_avg'] = df.runs.rolling(5).mean().shift()
            df['runs_allowed_moving_avg'] = df.runs_allowed.rolling(5).mean().shift()
            df.runs_allowed_moving_avg.fillna(df.iloc[5].runs_allowed_moving_avg, inplace=True)

            # win_pct of last ten games
            win_pct = [list(df.w_or_l[i-10:i]).count('W') / 10. for i in range(10, len(df))]
            for i in range(10):
                win_pct.insert(0, None)
            df['win_pct'] = win_pct

            # return home games only
            df = df[~df['at'].str.contains('@')].reset_index(drop=True)

            # make dummny variable if game was on opening day or not
            df['opening_day'] = [1 if x == min(df.date) else 0 for x in df.date]

            # concatenate the two dataframes 
            team_df = pd.concat([team_df, df]).reset_index(drop=True)

            #sleep
            sleep(0.5)

        except:
            pass

    # save it to csv
    team_df.to_csv('/Users/andreewerner/Desktop/mlb-game-data-MIL.csv', index=False, encoding='utf-8')
    return team_df
games()

,date,box,team,at,opponent,w_or_l,runs,runs_allowed,innings,WL_record,rank,gb,winning_pitcher,losing_pitcher,save,time,d_or_n,attendance,streak,double_header,runs_per_game,runs_moving_avg,runs_allowed_moving_avg,win_pct,opening_day
0,2000-04-10,boxscore,MIL,,FLA,W,4,3,,3-3,2,2.0,Haynes,Nunez,Wickman,3:04,D,"53,509",--,0,5.142857,5.6,6.4,NaN,1
1,2000-04-12,boxscore,MIL,,FLA,L,4,11,,4-3,2,2.0,Sanchez,Navarro,,3:29,N,"7,007",+,0,5.000000,4.8,6.0,NaN,0
2,2000-04-13,boxscore,MIL,,FLA,W,4,0,,4-4,2,2.5,Bere,Penny,,2:28,N,"7,023",-,0,4.888889,5.4,7.2,NaN,0
3,2000-04-14,boxscore,MIL,,ATL,L,3,6,,5-4,2,1.5,Glavine,Woodard,Remlinger,2:48,N,"10,171",+,0,4.800000,4.4,7.0,0.5,0
4,2000-04-15,boxscore,MIL,,ATL,W,6,3,,5-5,2,1.5,Weathers,Burkett,Wickman,2:24,D,"24,755",-,0,4.636364,3.4,6.2,0.5,0
5,2000-04-16,boxscore,MIL,,ATL,L,1,2,,6-5,2,1.0,Mulholland,Stull,Remlinger,2:31,D,"12,004",+,0,4.750000,4.2,4.6,0.5,0
6,2000-04-28,boxscore,MIL,,HOU,L,0,7,,8-13,3,5.5,Holt,Navarro,,2:49,N,"15,459",+,0,4.181818,4.2,5.4,0.2,0
7,2000-04-29,boxscore,MIL,,HOU,L,3,10,,8-14,6,6.5,Maddux,Stull,,3:32,D,"22,630",-,0,4.000000,2.8,6.2,0.2,0
8,2000-04-30,boxscore,MIL,,HOU,W-wo,4,3,,8-15,6,7.5,Weathers,Henry,,3:22,D,"19,647",--,0,3.958333,2.6,7.0,0.2,0
9,2000-05-01,boxscore,MIL,,HOU,L,0,5,,9-15,4,7.5,Dotel,Woodard,,3:02,N,"7,247",+,0,3.960000,3.0,6.2,0.2,0


In [121]:
def games():
    team_df = pd.DataFrame(columns=['date', 'box', 'team', 'at', 'opponent', 'w_or_l', 'runs', 'runs_allowed',
                                    'innings', 'WL_record', 'rank', 'gb', 'winning_pitcher', 'losing_pitcher',
                                    'save', 'time', 'd_or_n', 'attendance', 'streak', 'double_header', 'runs_per_game',
                                    'runs_moving_avg', 'runs_allowed_moving_avg', 'win_pct', 'opening_day'])
   # for team in teams:
    for year in range (2000, 2018):
        try:
            df = pd.DataFrame(columns=['date', 'box', 'team', 'at', 'opponent', 'w_or_l', 'runs', 'runs_allowed',
                                       'innings', 'WL_record', 'rank', 'gb', 'winning_pitcher', 'losing_pitcher',
                                       'save', 'time', 'd_or_n', 'attendance', 'streak'])
            urls = requests.get('http://www.baseball-reference.com/teams/' + 'MIN' + '/' + str(year) + '-schedule-scores.shtml')
            soup = BeautifulSoup(urls.text, 'html.parser')
            for game in soup.find('table', {'class':'stats_table'}).find_all('tr'):
                results = []
                for element in game.find_all('td'):
                    results.append(element.text)
                    if len(results) == 19:
                        df.loc[len(df)] = results



            # double header variable for when 2 teams play same day in front of same crowd
            double_header = '('
            df['double_header'] = [1 if double_header in x else 0 for x in df.date]

            # convert date column to datetime
            df.date = df.date.str.replace(r"\(.*\)","")
            date = []
            for day in df.date:
                split = day.split(', ')
                date.append(split[1] + ', ' + str(year))
            df.date = date
            df.date = pd.to_datetime(df.date)

            # shift WL_record, rank, gb, streak and fill in the first value
            df.WL_record = df.WL_record.shift()
            df.WL_record.fillna('0-0', inplace=True)

            for col in ['rank', 'gb', 'streak']:
                df[col] = df[col].shift()
                df[col].fillna('0', inplace=True)

            # rolling means for runs and runs allowed
            df['runs_per_game'] = [0 if x == min(df.date) else df[df.date < x].runs.astype(int).mean() for x in df.date]
            df['runs_moving_avg'] = df.runs.rolling(5).mean().shift()
            df['runs_allowed_moving_avg'] = df.runs_allowed.rolling(5).mean().shift()
            df.runs_allowed_moving_avg.fillna(df.iloc[5].runs_allowed_moving_avg, inplace=True)

            # win_pct of last ten games
            win_pct = [list(df.w_or_l[i-10:i]).count('W') / 10. for i in range(10, len(df))]
            for i in range(10):
                win_pct.insert(0, None)
            df['win_pct'] = win_pct

            # return home games only
            df = df[~df['at'].str.contains('@')].reset_index(drop=True)

            # make dummny variable if game was on opening day or not
            df['opening_day'] = [1 if x == min(df.date) else 0 for x in df.date]

            # concatenate the two dataframes 
            team_df = pd.concat([team_df, df]).reset_index(drop=True)

            #sleep
            sleep(0.5)

        except:
            pass

    # save it to csv
    team_df.to_csv('/Users/andreewerner/Desktop/mlb-game-data-MIN.csv', index=False, encoding='utf-8')
    return team_df
games()

,date,box,team,at,opponent,w_or_l,runs,runs_allowed,innings,WL_record,rank,gb,winning_pitcher,losing_pitcher,save,time,d_or_n,attendance,streak,double_header,runs_per_game,runs_moving_avg,runs_allowed_moving_avg,win_pct,opening_day
0,2000-04-03,boxscore,MIN,,TBD,L,0,7,,0-0,0,0,Trachsel,Radke,,2:50,N,"43,830",0,0,0.000000,NaN,7.2,NaN,1
1,2000-04-04,boxscore,MIN,,TBD,W-wo,6,5,,0-1,3,1.0,Carrasco,Hernandez,,2:50,N,"7,020",-,0,0.000000,NaN,7.2,NaN,0
2,2000-04-05,boxscore,MIN,,TBD,W-wo,10,7,,1-1,2,0.5,Guardado,White,,2:49,N,"7,089",+,0,3.000000,NaN,7.2,NaN,0
3,2000-04-06,boxscore,MIN,,TBD,L,6,7,,2-1,1,up 0.5,Mecir,Carrasco,Hernandez,2:44,D,"8,763",++,0,5.333333,NaN,7.2,NaN,0
4,2000-04-14,boxscore,MIN,,BAL,W,10,9,,3-8,5,5.0,Redman,Groom,Guardado,3:04,N,"15,249",----,0,5.272727,5.6,7.4,0.1,0
5,2000-04-15,boxscore,MIN,,BAL,L,4,6,,4-8,5,4.0,Worrell,Miller,Trombley,2:49,N,"18,745",+,0,5.666667,5.0,7.8,0.2,0
6,2000-04-16,boxscore,MIN,,BAL,L,0,5,,4-9,5,4.0,Ponson,Mays,,2:08,D,"14,066",-,0,5.538462,4.8,7.8,0.2,0
7,2000-04-18,boxscore,MIN,,KCR,W,3,1,,4-10,5,5.0,Radke,Suppan,Wells,2:23,N,"6,630",--,0,5.142857,4.0,6.2,0.2,0
8,2000-04-19,boxscore,MIN,,KCR,W-wo,7,6,,5-10,4,4.0,Carrasco,Santiago,,3:02,N,"7,606",+,0,5.000000,4.0,5.0,0.3,0
9,2000-04-20,boxscore,MIN,,KCR,W,9,7,,6-10,4,3.5,Bergman,Rosado,Wells,3:03,D,"9,213",++,0,5.125000,4.8,5.4,0.3,0


In [122]:
def games():
    team_df = pd.DataFrame(columns=['date', 'box', 'team', 'at', 'opponent', 'w_or_l', 'runs', 'runs_allowed',
                                    'innings', 'WL_record', 'rank', 'gb', 'winning_pitcher', 'losing_pitcher',
                                    'save', 'time', 'd_or_n', 'attendance', 'streak', 'double_header', 'runs_per_game',
                                    'runs_moving_avg', 'runs_allowed_moving_avg', 'win_pct', 'opening_day'])
   # for team in teams:
    for year in range (2000, 2018):
        try:
            df = pd.DataFrame(columns=['date', 'box', 'team', 'at', 'opponent', 'w_or_l', 'runs', 'runs_allowed',
                                       'innings', 'WL_record', 'rank', 'gb', 'winning_pitcher', 'losing_pitcher',
                                       'save', 'time', 'd_or_n', 'attendance', 'streak'])
            urls = requests.get('http://www.baseball-reference.com/teams/' + 'MON' + '/' + str(year) + '-schedule-scores.shtml')
            soup = BeautifulSoup(urls.text, 'html.parser')
            for game in soup.find('table', {'class':'stats_table'}).find_all('tr'):
                results = []
                for element in game.find_all('td'):
                    results.append(element.text)
                    if len(results) == 19:
                        df.loc[len(df)] = results



            # double header variable for when 2 teams play same day in front of same crowd
            double_header = '('
            df['double_header'] = [1 if double_header in x else 0 for x in df.date]

            # convert date column to datetime
            df.date = df.date.str.replace(r"\(.*\)","")
            date = []
            for day in df.date:
                split = day.split(', ')
                date.append(split[1] + ', ' + str(year))
            df.date = date
            df.date = pd.to_datetime(df.date)

            # shift WL_record, rank, gb, streak and fill in the first value
            df.WL_record = df.WL_record.shift()
            df.WL_record.fillna('0-0', inplace=True)

            for col in ['rank', 'gb', 'streak']:
                df[col] = df[col].shift()
                df[col].fillna('0', inplace=True)

            # rolling means for runs and runs allowed
            df['runs_per_game'] = [0 if x == min(df.date) else df[df.date < x].runs.astype(int).mean() for x in df.date]
            df['runs_moving_avg'] = df.runs.rolling(5).mean().shift()
            df['runs_allowed_moving_avg'] = df.runs_allowed.rolling(5).mean().shift()
            df.runs_allowed_moving_avg.fillna(df.iloc[5].runs_allowed_moving_avg, inplace=True)

            # win_pct of last ten games
            win_pct = [list(df.w_or_l[i-10:i]).count('W') / 10. for i in range(10, len(df))]
            for i in range(10):
                win_pct.insert(0, None)
            df['win_pct'] = win_pct

            # return home games only
            df = df[~df['at'].str.contains('@')].reset_index(drop=True)

            # make dummny variable if game was on opening day or not
            df['opening_day'] = [1 if x == min(df.date) else 0 for x in df.date]

            # concatenate the two dataframes 
            team_df = pd.concat([team_df, df]).reset_index(drop=True)

            #sleep
            sleep(0.5)

        except:
            pass

    # save it to csv
    team_df.to_csv('/Users/andreewerner/Desktop/mlb-game-data-MON.csv', index=False, encoding='utf-8')
    return team_df
games()

,date,box,team,at,opponent,w_or_l,runs,runs_allowed,innings,WL_record,rank,gb,winning_pitcher,losing_pitcher,save,time,d_or_n,attendance,streak,double_header,runs_per_game,runs_moving_avg,runs_allowed_moving_avg,win_pct,opening_day
0,2000-04-03,boxscore,MON,,LAD,L,4,10,,0-0,0,0,Brown,Hermanson,Adams,2:37,N,"51,249",0,0,0.000000,NaN,7.6,NaN,1
1,2000-04-04,boxscore,MON,,LAD,L,4,10,,0-1,5,1.0,Park,Irabu,,2:59,N,"12,143",-,0,4.000000,NaN,7.6,NaN,0
2,2000-04-05,boxscore,MON,,LAD,W-wo,6,5,,0-2,5,1.5,Telford,Shaw,,2:58,N,"8,867",--,0,4.000000,NaN,7.6,NaN,0
3,2000-04-06,boxscore,MON,,LAD,W,11,3,,1-2,3,1.0,Pavano,Perez,,2:27,N,"9,121",+,0,4.666667,NaN,7.6,NaN,0
4,2000-04-07,boxscore,MON,,SDP,L,5,10,,2-2,2,0.5,Meadows,Powell,,2:55,N,"12,260",++,0,6.250000,NaN,7.6,NaN,0
5,2000-04-08,boxscore,MON,,SDP,W,10,9,,2-3,4,1.0,Telford,Wall,Urbina,2:49,D,"13,528",-,0,6.000000,6.0,7.6,NaN,0
6,2000-04-09,boxscore,MON,,SDP,W,2,1,,3-3,2,0.5,Irabu,Boehringer,Urbina,2:13,D,"9,782",+,0,6.666667,7.2,7.4,NaN,0
7,2000-04-18,boxscore,MON,,CHC,W,4,3,,6-6,2,0.5,Strickland,Tapani,Urbina,2:23,N,"9,975",-,0,5.333333,4.4,3.6,0.5,0
8,2000-04-19,boxscore,MON,,CHC,W,7,3,,7-6,2,1.0,Pavano,Quevedo,,2:42,N,"10,112",+,0,5.230769,3.8,3.6,0.6,0
9,2000-04-20,boxscore,MON,,CHC,L,6,10,,8-6,1,Tied,Williams,Blank,,2:54,N,"12,186",++,0,5.357143,4.4,3.0,0.6,0


In [123]:
def games():
    team_df = pd.DataFrame(columns=['date', 'box', 'team', 'at', 'opponent', 'w_or_l', 'runs', 'runs_allowed',
                                    'innings', 'WL_record', 'rank', 'gb', 'winning_pitcher', 'losing_pitcher',
                                    'save', 'time', 'd_or_n', 'attendance', 'streak', 'double_header', 'runs_per_game',
                                    'runs_moving_avg', 'runs_allowed_moving_avg', 'win_pct', 'opening_day'])
   # for team in teams:
    for year in range (2000, 2018):
        try:
            df = pd.DataFrame(columns=['date', 'box', 'team', 'at', 'opponent', 'w_or_l', 'runs', 'runs_allowed',
                                       'innings', 'WL_record', 'rank', 'gb', 'winning_pitcher', 'losing_pitcher',
                                       'save', 'time', 'd_or_n', 'attendance', 'streak'])
            urls = requests.get('http://www.baseball-reference.com/teams/' + 'NYM' + '/' + str(year) + '-schedule-scores.shtml')
            soup = BeautifulSoup(urls.text, 'html.parser')
            for game in soup.find('table', {'class':'stats_table'}).find_all('tr'):
                results = []
                for element in game.find_all('td'):
                    results.append(element.text)
                    if len(results) == 19:
                        df.loc[len(df)] = results



            # double header variable for when 2 teams play same day in front of same crowd
            double_header = '('
            df['double_header'] = [1 if double_header in x else 0 for x in df.date]

            # convert date column to datetime
            df.date = df.date.str.replace(r"\(.*\)","")
            date = []
            for day in df.date:
                split = day.split(', ')
                date.append(split[1] + ', ' + str(year))
            df.date = date
            df.date = pd.to_datetime(df.date)

            # shift WL_record, rank, gb, streak and fill in the first value
            df.WL_record = df.WL_record.shift()
            df.WL_record.fillna('0-0', inplace=True)

            for col in ['rank', 'gb', 'streak']:
                df[col] = df[col].shift()
                df[col].fillna('0', inplace=True)

            # rolling means for runs and runs allowed
            df['runs_per_game'] = [0 if x == min(df.date) else df[df.date < x].runs.astype(int).mean() for x in df.date]
            df['runs_moving_avg'] = df.runs.rolling(5).mean().shift()
            df['runs_allowed_moving_avg'] = df.runs_allowed.rolling(5).mean().shift()
            df.runs_allowed_moving_avg.fillna(df.iloc[5].runs_allowed_moving_avg, inplace=True)

            # win_pct of last ten games
            win_pct = [list(df.w_or_l[i-10:i]).count('W') / 10. for i in range(10, len(df))]
            for i in range(10):
                win_pct.insert(0, None)
            df['win_pct'] = win_pct

            # return home games only
            df = df[~df['at'].str.contains('@')].reset_index(drop=True)

            # make dummny variable if game was on opening day or not
            df['opening_day'] = [1 if x == min(df.date) else 0 for x in df.date]

            # concatenate the two dataframes 
            team_df = pd.concat([team_df, df]).reset_index(drop=True)

            #sleep
            sleep(0.5)

        except:
            pass

    # save it to csv
    team_df.to_csv('/Users/andreewerner/Desktop/mlb-game-data-NYM.csv', index=False, encoding='utf-8')
    return team_df
games()

,date,box,team,at,opponent,w_or_l,runs,runs_allowed,innings,WL_record,rank,gb,winning_pitcher,losing_pitcher,save,time,d_or_n,attendance,streak,double_header,runs_per_game,runs_moving_avg,runs_allowed_moving_avg,win_pct,opening_day
0,2000-03-29,boxscore,NYM,,CHC,L,3,5,,0-0,0,0,Lieber,Hampton,Aguilera,3:26,N,"55,000",0,0,0.000000,NaN,3.8,NaN,1
1,2000-04-03,boxscore,NYM,,SDP,W,2,1,,1-1,1,Tied,Leiter,Wall,Benitez,2:24,D,"52,308",+,0,4.000000,NaN,3.8,NaN,0
2,2000-04-05,boxscore,NYM,,SDP,L,0,4,,2-1,1,Tied,Williams,Jones,,3:04,N,"17,585",++,0,3.333333,NaN,3.8,NaN,0
3,2000-04-06,boxscore,NYM,,SDP,L,5,8,,2-2,2,0.5,Clement,Hampton,,3:22,N,"17,482",-,0,2.500000,NaN,3.8,NaN,0
4,2000-04-07,boxscore,NYM,,LAD,W,2,1,,2-3,4,1.0,Reed,Dreifort,Benitez,2:33,N,"27,791",--,0,3.000000,3.0,3.8,NaN,0
5,2000-04-08,boxscore,NYM,,LAD,L,5,6,10,3-3,2,0.5,Fetters,Benitez,Shaw,3:42,D,"37,814",+,0,2.833333,2.8,3.0,NaN,0
6,2000-04-18,boxscore,NYM,,MIL,W,10,7,,6-7,4,1.0,Hampton,Navarro,Benitez,3:17,N,"17,905",+,0,4.307692,5.4,5.0,0.4,0
7,2000-04-19,boxscore,NYM,,MIL,W,3,1,,7-7,4,1.5,Reed,Bere,Benitez,3:16,N,"18,001",++,0,4.714286,6.4,4.8,0.5,0
8,2000-04-20,boxscore,NYM,,MIL,W-wo,5,4,10,8-7,4,0.5,Wendell,Leskanic,,3:29,N,"17,002",+++,0,4.600000,6.6,4.8,0.6,0
9,2000-04-22,boxscore,NYM,,CHC,W,8,3,,9-7,3,0.5,Rusch,Lieber,,2:48,D,,++++,1,4.625000,6.0,4.6,0.5,0


In [124]:
def games():
    team_df = pd.DataFrame(columns=['date', 'box', 'team', 'at', 'opponent', 'w_or_l', 'runs', 'runs_allowed',
                                    'innings', 'WL_record', 'rank', 'gb', 'winning_pitcher', 'losing_pitcher',
                                    'save', 'time', 'd_or_n', 'attendance', 'streak', 'double_header', 'runs_per_game',
                                    'runs_moving_avg', 'runs_allowed_moving_avg', 'win_pct', 'opening_day'])
   # for team in teams:
    for year in range (2000, 2018):
        try:
            df = pd.DataFrame(columns=['date', 'box', 'team', 'at', 'opponent', 'w_or_l', 'runs', 'runs_allowed',
                                       'innings', 'WL_record', 'rank', 'gb', 'winning_pitcher', 'losing_pitcher',
                                       'save', 'time', 'd_or_n', 'attendance', 'streak'])
            urls = requests.get('http://www.baseball-reference.com/teams/' + 'NYY' + '/' + str(year) + '-schedule-scores.shtml')
            soup = BeautifulSoup(urls.text, 'html.parser')
            for game in soup.find('table', {'class':'stats_table'}).find_all('tr'):
                results = []
                for element in game.find_all('td'):
                    results.append(element.text)
                    if len(results) == 19:
                        df.loc[len(df)] = results



            # double header variable for when 2 teams play same day in front of same crowd
            double_header = '('
            df['double_header'] = [1 if double_header in x else 0 for x in df.date]

            # convert date column to datetime
            df.date = df.date.str.replace(r"\(.*\)","")
            date = []
            for day in df.date:
                split = day.split(', ')
                date.append(split[1] + ', ' + str(year))
            df.date = date
            df.date = pd.to_datetime(df.date)

            # shift WL_record, rank, gb, streak and fill in the first value
            df.WL_record = df.WL_record.shift()
            df.WL_record.fillna('0-0', inplace=True)

            for col in ['rank', 'gb', 'streak']:
                df[col] = df[col].shift()
                df[col].fillna('0', inplace=True)

            # rolling means for runs and runs allowed
            df['runs_per_game'] = [0 if x == min(df.date) else df[df.date < x].runs.astype(int).mean() for x in df.date]
            df['runs_moving_avg'] = df.runs.rolling(5).mean().shift()
            df['runs_allowed_moving_avg'] = df.runs_allowed.rolling(5).mean().shift()
            df.runs_allowed_moving_avg.fillna(df.iloc[5].runs_allowed_moving_avg, inplace=True)

            # win_pct of last ten games
            win_pct = [list(df.w_or_l[i-10:i]).count('W') / 10. for i in range(10, len(df))]
            for i in range(10):
                win_pct.insert(0, None)
            df['win_pct'] = win_pct

            # return home games only
            df = df[~df['at'].str.contains('@')].reset_index(drop=True)

            # make dummny variable if game was on opening day or not
            df['opening_day'] = [1 if x == min(df.date) else 0 for x in df.date]

            # concatenate the two dataframes 
            team_df = pd.concat([team_df, df]).reset_index(drop=True)

            #sleep
            sleep(0.5)

        except:
            pass

    # save it to csv
    team_df.to_csv('/Users/andreewerner/Desktop/mlb-game-data-NYY.csv', index=False, encoding='utf-8')
    return team_df
games()

,date,box,team,at,opponent,w_or_l,runs,runs_allowed,innings,WL_record,rank,gb,winning_pitcher,losing_pitcher,save,time,d_or_n,attendance,streak,double_header,runs_per_game,runs_moving_avg,runs_allowed_moving_avg,win_pct,opening_day
0,2000-04-12,boxscore,NYY,,TEX,W,8,6,,3-3,2,2.0,Nelson,Munoz,Rivera,3:33,D,"48,487",-,0,4.166667,4.4,6.6,NaN,1
1,2000-04-13,boxscore,NYY,,TEX,W,5,1,,4-3,2,0.5,Hernandez,Rogers,,2:59,N,"23,805",+,0,4.714286,5.0,7.2,NaN,0
2,2000-04-14,boxscore,NYY,,KCR,W,7,5,,5-3,1,up 0.5,Clemens,Witasick,Rivera,3:48,N,"33,094",++,0,4.750000,4.8,5.0,NaN,0
3,2000-04-15,boxscore,NYY,,KCR,W,7,1,,6-3,1,up 1.5,Mendoza,Rosado,,2:33,D,"34,056",+++,0,5.000000,5.2,4.6,NaN,0
4,2000-04-16,boxscore,NYY,,KCR,W,8,4,,7-3,1,up 1.5,Nelson,Fussell,,3:02,D,"36,724",++++,0,5.200000,6.0,4.4,0.7,0
5,2000-04-24,boxscore,NYY,,MIN,L,3,7,,12-5,1,up 1.0,Milton,Clemens,,3:10,D,"35,693",+,0,5.352941,5.2,6.0,0.8,0
6,2000-04-25,boxscore,NYY,,MIN,L,1,6,,12-6,1,up 1.0,Bergman,Mendoza,,3:00,N,"27,372",-,0,5.222222,4.6,6.8,0.7,0
7,2000-04-26,boxscore,NYY,,MIN,W,2,0,,12-7,1,Tied,Nelson,Mays,Rivera,2:34,D,"27,919",--,0,5.000000,3.8,7.2,0.6,0
8,2000-04-28,boxscore,NYY,,TOR,W,6,0,,13-7,1,up 1.0,Cone,Escobar,,2:47,N,"35,987",+,0,4.850000,3.6,5.6,0.6,0
9,2000-04-29,boxscore,NYY,,TOR,L,2,6,,14-7,1,up 1.5,Wells,Hernandez,Koch,3:23,D,"38,783",++,0,4.904762,4.4,4.0,0.6,0


In [125]:
def games():
    team_df = pd.DataFrame(columns=['date', 'box', 'team', 'at', 'opponent', 'w_or_l', 'runs', 'runs_allowed',
                                    'innings', 'WL_record', 'rank', 'gb', 'winning_pitcher', 'losing_pitcher',
                                    'save', 'time', 'd_or_n', 'attendance', 'streak', 'double_header', 'runs_per_game',
                                    'runs_moving_avg', 'runs_allowed_moving_avg', 'win_pct', 'opening_day'])
   # for team in teams:
    for year in range (2000, 2018):
        try:
            df = pd.DataFrame(columns=['date', 'box', 'team', 'at', 'opponent', 'w_or_l', 'runs', 'runs_allowed',
                                       'innings', 'WL_record', 'rank', 'gb', 'winning_pitcher', 'losing_pitcher',
                                       'save', 'time', 'd_or_n', 'attendance', 'streak'])
            urls = requests.get('http://www.baseball-reference.com/teams/' + 'OAK' + '/' + str(year) + '-schedule-scores.shtml')
            soup = BeautifulSoup(urls.text, 'html.parser')
            for game in soup.find('table', {'class':'stats_table'}).find_all('tr'):
                results = []
                for element in game.find_all('td'):
                    results.append(element.text)
                    if len(results) == 19:
                        df.loc[len(df)] = results



            # double header variable for when 2 teams play same day in front of same crowd
            double_header = '('
            df['double_header'] = [1 if double_header in x else 0 for x in df.date]

            # convert date column to datetime
            df.date = df.date.str.replace(r"\(.*\)","")
            date = []
            for day in df.date:
                split = day.split(', ')
                date.append(split[1] + ', ' + str(year))
            df.date = date
            df.date = pd.to_datetime(df.date)

            # shift WL_record, rank, gb, streak and fill in the first value
            df.WL_record = df.WL_record.shift()
            df.WL_record.fillna('0-0', inplace=True)

            for col in ['rank', 'gb', 'streak']:
                df[col] = df[col].shift()
                df[col].fillna('0', inplace=True)

            # rolling means for runs and runs allowed
            df['runs_per_game'] = [0 if x == min(df.date) else df[df.date < x].runs.astype(int).mean() for x in df.date]
            df['runs_moving_avg'] = df.runs.rolling(5).mean().shift()
            df['runs_allowed_moving_avg'] = df.runs_allowed.rolling(5).mean().shift()
            df.runs_allowed_moving_avg.fillna(df.iloc[5].runs_allowed_moving_avg, inplace=True)

            # win_pct of last ten games
            win_pct = [list(df.w_or_l[i-10:i]).count('W') / 10. for i in range(10, len(df))]
            for i in range(10):
                win_pct.insert(0, None)
            df['win_pct'] = win_pct

            # return home games only
            df = df[~df['at'].str.contains('@')].reset_index(drop=True)

            # make dummny variable if game was on opening day or not
            df['opening_day'] = [1 if x == min(df.date) else 0 for x in df.date]

            # concatenate the two dataframes 
            team_df = pd.concat([team_df, df]).reset_index(drop=True)

            #sleep
            sleep(0.5)

        except:
            pass

    # save it to csv
    team_df.to_csv('/Users/andreewerner/Desktop/mlb-game-data-OAK.csv', index=False, encoding='utf-8')
    return team_df
games()

,date,box,team,at,opponent,w_or_l,runs,runs_allowed,innings,WL_record,rank,gb,winning_pitcher,losing_pitcher,save,time,d_or_n,attendance,streak,double_header,runs_per_game,runs_moving_avg,runs_allowed_moving_avg,win_pct,opening_day
0,2000-04-03,boxscore,OAK,,DET,L,4,7,,0-0,0,0,Nomo,Appier,Jones,3:13,N,"53,498",0,0,0.000000,NaN,4.8,NaN,1
1,2000-04-04,boxscore,OAK,,DET,W,3,1,,0-1,3,1.0,Hudson,Nitkowski,Isringhausen,2:39,N,"10,188",-,0,4.000000,NaN,4.8,NaN,0
2,2000-04-05,boxscore,OAK,,DET,W,8,2,,1-1,2,1.0,Olivares,Moehler,,2:42,D,"10,229",+,0,3.500000,NaN,4.8,NaN,0
3,2000-04-07,boxscore,OAK,,CHW,L,6,7,,2-1,1,Tied,Eyre,Magnante,Foulke,3:36,N,"10,549",++,0,5.000000,NaN,4.8,NaN,0
4,2000-04-08,boxscore,OAK,,CHW,L,3,7,,2-2,3,1.0,Sirotka,Mahay,,2:59,D,"13,125",-,0,5.250000,NaN,4.8,NaN,0
5,2000-04-09,boxscore,OAK,,CHW,W,14,2,,2-3,4,1.0,Appier,Wells,,3:00,D,"13,280",--,0,4.800000,4.8,4.8,NaN,0
6,2000-04-10,boxscore,OAK,,CLE,L,4,9,,3-3,3,1.0,Kamieniecki,Tam,,3:52,N,"9,123",+,0,6.333333,6.8,3.8,NaN,0
7,2000-04-11,boxscore,OAK,,CLE,L,1,5,,3-4,4,1.5,Nagy,Olivares,,2:58,N,"8,638",-,0,6.000000,7.0,5.4,NaN,0
8,2000-04-12,boxscore,OAK,,CLE,L,0,5,,3-5,4,2.0,Wright,Heredia,,2:33,D,"14,683",--,0,5.375000,5.6,6.0,NaN,0
9,2000-04-21,boxscore,OAK,,BAL,L,9,11,,7-9,3,2.0,Ponson,Olivares,Timlin,3:26,N,"11,802",-,0,5.375000,5.6,4.8,0.4,0


In [126]:
def games():
    team_df = pd.DataFrame(columns=['date', 'box', 'team', 'at', 'opponent', 'w_or_l', 'runs', 'runs_allowed',
                                    'innings', 'WL_record', 'rank', 'gb', 'winning_pitcher', 'losing_pitcher',
                                    'save', 'time', 'd_or_n', 'attendance', 'streak', 'double_header', 'runs_per_game',
                                    'runs_moving_avg', 'runs_allowed_moving_avg', 'win_pct', 'opening_day'])
   # for team in teams:
    for year in range (2000, 2018):
        try:
            df = pd.DataFrame(columns=['date', 'box', 'team', 'at', 'opponent', 'w_or_l', 'runs', 'runs_allowed',
                                       'innings', 'WL_record', 'rank', 'gb', 'winning_pitcher', 'losing_pitcher',
                                       'save', 'time', 'd_or_n', 'attendance', 'streak'])
            urls = requests.get('http://www.baseball-reference.com/teams/' + 'PHI' + '/' + str(year) + '-schedule-scores.shtml')
            soup = BeautifulSoup(urls.text, 'html.parser')
            for game in soup.find('table', {'class':'stats_table'}).find_all('tr'):
                results = []
                for element in game.find_all('td'):
                    results.append(element.text)
                    if len(results) == 19:
                        df.loc[len(df)] = results



            # double header variable for when 2 teams play same day in front of same crowd
            double_header = '('
            df['double_header'] = [1 if double_header in x else 0 for x in df.date]

            # convert date column to datetime
            df.date = df.date.str.replace(r"\(.*\)","")
            date = []
            for day in df.date:
                split = day.split(', ')
                date.append(split[1] + ', ' + str(year))
            df.date = date
            df.date = pd.to_datetime(df.date)

            # shift WL_record, rank, gb, streak and fill in the first value
            df.WL_record = df.WL_record.shift()
            df.WL_record.fillna('0-0', inplace=True)

            for col in ['rank', 'gb', 'streak']:
                df[col] = df[col].shift()
                df[col].fillna('0', inplace=True)

            # rolling means for runs and runs allowed
            df['runs_per_game'] = [0 if x == min(df.date) else df[df.date < x].runs.astype(int).mean() for x in df.date]
            df['runs_moving_avg'] = df.runs.rolling(5).mean().shift()
            df['runs_allowed_moving_avg'] = df.runs_allowed.rolling(5).mean().shift()
            df.runs_allowed_moving_avg.fillna(df.iloc[5].runs_allowed_moving_avg, inplace=True)

            # win_pct of last ten games
            win_pct = [list(df.w_or_l[i-10:i]).count('W') / 10. for i in range(10, len(df))]
            for i in range(10):
                win_pct.insert(0, None)
            df['win_pct'] = win_pct

            # return home games only
            df = df[~df['at'].str.contains('@')].reset_index(drop=True)

            # make dummny variable if game was on opening day or not
            df['opening_day'] = [1 if x == min(df.date) else 0 for x in df.date]

            # concatenate the two dataframes 
            team_df = pd.concat([team_df, df]).reset_index(drop=True)

            #sleep
            sleep(0.5)

        except:
            pass

    # save it to csv
    team_df.to_csv('/Users/andreewerner/Desktop/mlb-game-data-PHI.csv', index=False, encoding='utf-8')
    return team_df
games()

,date,box,team,at,opponent,w_or_l,runs,runs_allowed,innings,WL_record,rank,gb,winning_pitcher,losing_pitcher,save,time,d_or_n,attendance,streak,double_header,runs_per_game,runs_moving_avg,runs_allowed_moving_avg,win_pct,opening_day
0,2000-04-10,boxscore,PHI,,NYM,W,9,7,,2-4,5,2.0,Telemaco,Rodriguez,Gomes,3:02,N,"47,115",+,0,3.500000,3.4,5.0,NaN,1
1,2000-04-12,boxscore,PHI,,NYM,W,8,5,,3-4,4,1.0,Person,Hampton,Gomes,3:09,N,"15,269",++,0,4.285714,4.6,4.2,NaN,0
2,2000-04-13,boxscore,PHI,,NYM,L,1,2,,4-4,3,0.5,Cook,Aldred,Benitez,3:18,N,"14,552",+++,0,4.750000,5.8,4.6,NaN,0
3,2000-04-14,boxscore,PHI,,MON,L,0,4,,4-5,3,0.5,Hermanson,Brock,,2:29,N,"12,366",-,0,4.333333,5.2,4.8,NaN,0
4,2000-04-16,boxscore,PHI,,MON,W-wo,5,4,,4-6,5,1.5,Aldred,Telford,,2:46,D,"18,648",--,0,3.900000,4.2,4.0,0.4,0
5,2000-04-25,boxscore,PHI,,ARI,L,2,10,,6-12,5,6.0,Johnson,Brock,,2:52,N,"11,926",---,0,3.388889,2.6,4.2,0.1,0
6,2000-04-26,boxscore,PHI,,ARI,L,4,10,,6-13,5,7.0,Stottlemyre,Ashby,,3:02,N,"12,250",----,0,3.315789,2.2,5.0,0.1,0
7,2000-04-27,boxscore,PHI,,ARI,W-wo,5,4,,6-14,5,8.0,Gomes,Kim,,3:01,D,"13,560",-----,0,3.350000,2.2,6.4,0.1,0
8,2000-04-28,boxscore,PHI,,STL,L,4,7,,7-14,5,8.0,Stephenson,Reyes,Veres,3:11,N,"25,877",+,0,3.428571,2.8,6.4,0.1,0
9,2000-04-29,boxscore,PHI,,STL,L,6,7,10,7-15,5,9.0,Mohler,Gomes,,3:32,N,"23,031",-,0,3.454545,3.2,6.8,0.1,0


In [127]:
def games():
    team_df = pd.DataFrame(columns=['date', 'box', 'team', 'at', 'opponent', 'w_or_l', 'runs', 'runs_allowed',
                                    'innings', 'WL_record', 'rank', 'gb', 'winning_pitcher', 'losing_pitcher',
                                    'save', 'time', 'd_or_n', 'attendance', 'streak', 'double_header', 'runs_per_game',
                                    'runs_moving_avg', 'runs_allowed_moving_avg', 'win_pct', 'opening_day'])
   # for team in teams:
    for year in range (2000, 2018):
        try:
            df = pd.DataFrame(columns=['date', 'box', 'team', 'at', 'opponent', 'w_or_l', 'runs', 'runs_allowed',
                                       'innings', 'WL_record', 'rank', 'gb', 'winning_pitcher', 'losing_pitcher',
                                       'save', 'time', 'd_or_n', 'attendance', 'streak'])
            urls = requests.get('http://www.baseball-reference.com/teams/' + 'PIT' + '/' + str(year) + '-schedule-scores.shtml')
            soup = BeautifulSoup(urls.text, 'html.parser')
            for game in soup.find('table', {'class':'stats_table'}).find_all('tr'):
                results = []
                for element in game.find_all('td'):
                    results.append(element.text)
                    if len(results) == 19:
                        df.loc[len(df)] = results



            # double header variable for when 2 teams play same day in front of same crowd
            double_header = '('
            df['double_header'] = [1 if double_header in x else 0 for x in df.date]

            # convert date column to datetime
            df.date = df.date.str.replace(r"\(.*\)","")
            date = []
            for day in df.date:
                split = day.split(', ')
                date.append(split[1] + ', ' + str(year))
            df.date = date
            df.date = pd.to_datetime(df.date)

            # shift WL_record, rank, gb, streak and fill in the first value
            df.WL_record = df.WL_record.shift()
            df.WL_record.fillna('0-0', inplace=True)

            for col in ['rank', 'gb', 'streak']:
                df[col] = df[col].shift()
                df[col].fillna('0', inplace=True)

            # rolling means for runs and runs allowed
            df['runs_per_game'] = [0 if x == min(df.date) else df[df.date < x].runs.astype(int).mean() for x in df.date]
            df['runs_moving_avg'] = df.runs.rolling(5).mean().shift()
            df['runs_allowed_moving_avg'] = df.runs_allowed.rolling(5).mean().shift()
            df.runs_allowed_moving_avg.fillna(df.iloc[5].runs_allowed_moving_avg, inplace=True)

            # win_pct of last ten games
            win_pct = [list(df.w_or_l[i-10:i]).count('W') / 10. for i in range(10, len(df))]
            for i in range(10):
                win_pct.insert(0, None)
            df['win_pct'] = win_pct

            # return home games only
            df = df[~df['at'].str.contains('@')].reset_index(drop=True)

            # make dummny variable if game was on opening day or not
            df['opening_day'] = [1 if x == min(df.date) else 0 for x in df.date]

            # concatenate the two dataframes 
            team_df = pd.concat([team_df, df]).reset_index(drop=True)

            #sleep
            sleep(0.5)

        except:
            pass

    # save it to csv
    team_df.to_csv('/Users/andreewerner/Desktop/mlb-game-data-PIT.csv', index=False, encoding='utf-8')
    return team_df
games()

,date,box,team,at,opponent,w_or_l,runs,runs_allowed,innings,WL_record,rank,gb,winning_pitcher,losing_pitcher,save,time,d_or_n,attendance,streak,double_header,runs_per_game,runs_moving_avg,runs_allowed_moving_avg,win_pct,opening_day
0,2000-04-04,boxscore,PIT,,HOU,L,2,5,,0-0,0,0,Reynolds,Schmidt,Wagner,2:51,N,"54,399",0,0,0.000000,NaN,5.0,NaN,1
1,2000-04-05,boxscore,PIT,,HOU,L,2,11,,0-1,4,1.0,Lima,Benson,,2:32,N,"12,208",-,0,2.000000,NaN,5.0,NaN,0
2,2000-04-06,boxscore,PIT,,HOU,W,10,1,,0-2,4,2.0,Cordova,Holt,,2:44,D,"11,860",--,0,2.000000,NaN,5.0,NaN,0
3,2000-04-11,boxscore,PIT,,MON,L,3,7,,2-4,5,3.0,Vazquez,Benson,Kline,2:15,N,"11,335",--,0,4.333333,4.8,4.2,NaN,0
4,2000-04-12,boxscore,PIT,,MON,W,6,4,,2-5,5,4.5,Silva,Batista,Williams,2:58,N,"10,290",---,0,4.142857,5.0,3.4,NaN,0
5,2000-04-13,boxscore,PIT,,MON,W-wo,4,3,,3-5,5,3.5,Silva,Urbina,,2:23,N,"11,162",+,0,4.375000,4.2,4.0,NaN,0
6,2000-04-14,boxscore,PIT,,NYM,L,5,8,12,4-5,3,2.5,Franco,Silva,,4:10,N,"20,725",++,0,4.333333,3.6,4.2,NaN,0
7,2000-04-15,boxscore,PIT,,NYM,W,2,0,,4-6,4,2.5,Anderson,Rusch,Williams,1:57,D,"19,592",-,0,4.400000,3.6,4.6,0.3,0
8,2000-04-16,boxscore,PIT,,NYM,L,9,12,,5-6,3,2.0,Mahomes,Peters,Benitez,3:41,D,"20,724",+,0,4.181818,4.0,4.4,0.4,0
9,2000-04-25,boxscore,PIT,,SDP,W-wo,4,3,11,6-12,5,5.5,Sauerbeck,Almanzar,,3:30,N,"12,295",----,0,4.111111,2.8,3.8,0.2,0


In [128]:
def games():
    team_df = pd.DataFrame(columns=['date', 'box', 'team', 'at', 'opponent', 'w_or_l', 'runs', 'runs_allowed',
                                    'innings', 'WL_record', 'rank', 'gb', 'winning_pitcher', 'losing_pitcher',
                                    'save', 'time', 'd_or_n', 'attendance', 'streak', 'double_header', 'runs_per_game',
                                    'runs_moving_avg', 'runs_allowed_moving_avg', 'win_pct', 'opening_day'])
   # for team in teams:
    for year in range (2000, 2018):
        try:
            df = pd.DataFrame(columns=['date', 'box', 'team', 'at', 'opponent', 'w_or_l', 'runs', 'runs_allowed',
                                       'innings', 'WL_record', 'rank', 'gb', 'winning_pitcher', 'losing_pitcher',
                                       'save', 'time', 'd_or_n', 'attendance', 'streak'])
            urls = requests.get('http://www.baseball-reference.com/teams/' + 'SDP' + '/' + str(year) + '-schedule-scores.shtml')
            soup = BeautifulSoup(urls.text, 'html.parser')
            for game in soup.find('table', {'class':'stats_table'}).find_all('tr'):
                results = []
                for element in game.find_all('td'):
                    results.append(element.text)
                    if len(results) == 19:
                        df.loc[len(df)] = results



            # double header variable for when 2 teams play same day in front of same crowd
            double_header = '('
            df['double_header'] = [1 if double_header in x else 0 for x in df.date]

            # convert date column to datetime
            df.date = df.date.str.replace(r"\(.*\)","")
            date = []
            for day in df.date:
                split = day.split(', ')
                date.append(split[1] + ', ' + str(year))
            df.date = date
            df.date = pd.to_datetime(df.date)

            # shift WL_record, rank, gb, streak and fill in the first value
            df.WL_record = df.WL_record.shift()
            df.WL_record.fillna('0-0', inplace=True)

            for col in ['rank', 'gb', 'streak']:
                df[col] = df[col].shift()
                df[col].fillna('0', inplace=True)

            # rolling means for runs and runs allowed
            df['runs_per_game'] = [0 if x == min(df.date) else df[df.date < x].runs.astype(int).mean() for x in df.date]
            df['runs_moving_avg'] = df.runs.rolling(5).mean().shift()
            df['runs_allowed_moving_avg'] = df.runs_allowed.rolling(5).mean().shift()
            df.runs_allowed_moving_avg.fillna(df.iloc[5].runs_allowed_moving_avg, inplace=True)

            # win_pct of last ten games
            win_pct = [list(df.w_or_l[i-10:i]).count('W') / 10. for i in range(10, len(df))]
            for i in range(10):
                win_pct.insert(0, None)
            df['win_pct'] = win_pct

            # return home games only
            df = df[~df['at'].str.contains('@')].reset_index(drop=True)

            # make dummny variable if game was on opening day or not
            df['opening_day'] = [1 if x == min(df.date) else 0 for x in df.date]

            # concatenate the two dataframes 
            team_df = pd.concat([team_df, df]).reset_index(drop=True)

            #sleep
            sleep(0.5)

        except:
            pass

    # save it to csv
    team_df.to_csv('/Users/andreewerner/Desktop/mlb-game-data-SDP.csv', index=False, encoding='utf-8')
    return team_df
games()

,date,box,team,at,opponent,w_or_l,runs,runs_allowed,innings,WL_record,rank,gb,winning_pitcher,losing_pitcher,save,time,d_or_n,attendance,streak,double_header,runs_per_game,runs_moving_avg,runs_allowed_moving_avg,win_pct,opening_day
0,2000-04-10,boxscore,SDP,,ARI,L,4,8,,3-3,2,2.0,Stottlemyre,Williams,Morgan,2:56,D,"60,021",--,0,5.500000,6.4,4.4,NaN,1
1,2000-04-11,boxscore,SDP,,ARI,W-wo,3,2,13,3-4,3,3.0,Whisenant,Springer,,3:34,N,"19,035",---,0,5.285714,6.4,6.0,NaN,0
2,2000-04-12,boxscore,SDP,,ARI,W,4,2,,4-4,3,2.0,Meadows,Reynoso,Hoffman,2:24,N,"15,801",+,0,5.000000,5.4,5.4,NaN,0
3,2000-04-13,boxscore,SDP,,ARI,L,4,5,,5-4,3,1.0,Anderson,Hitchcock,Morgan,2:45,D,"28,131",++,0,4.888889,4.2,4.8,NaN,0
4,2000-04-14,boxscore,SDP,,HOU,L,4,10,,5-5,3,2.0,Reynolds,Boehringer,,3:02,N,"30,951",-,0,4.800000,3.2,3.8,0.4,0
5,2000-04-15,boxscore,SDP,,HOU,W,5,3,,5-6,4,3.0,Williams,Lima,Hoffman,2:21,N,"45,242",--,0,4.727273,3.8,5.4,0.4,0
6,2000-04-16,boxscore,SDP,,HOU,W,13,3,,6-6,4,3.0,Clement,Holt,,3:07,D,"46,352",+,0,4.750000,4.0,4.4,0.4,0
7,2000-04-28,boxscore,SDP,,ATL,L,2,7,,11-11,3,3.0,Maddux,Meadows,Remlinger,2:35,N,"33,995",+,0,5.772727,8.4,6.6,0.5,0
8,2000-04-29,boxscore,SDP,,ATL,L,4,7,12,11-12,3,3.0,Chen,Palacios,Rocker,4:06,N,"49,428",-,0,5.608696,7.2,6.8,0.4,0
9,2000-04-30,boxscore,SDP,,ATL,L,4,7,,11-13,3,4.0,Glavine,Hitchcock,,3:07,N,"44,845",--,0,5.541667,5.8,6.2,0.4,0


In [129]:
def games():
    team_df = pd.DataFrame(columns=['date', 'box', 'team', 'at', 'opponent', 'w_or_l', 'runs', 'runs_allowed',
                                    'innings', 'WL_record', 'rank', 'gb', 'winning_pitcher', 'losing_pitcher',
                                    'save', 'time', 'd_or_n', 'attendance', 'streak', 'double_header', 'runs_per_game',
                                    'runs_moving_avg', 'runs_allowed_moving_avg', 'win_pct', 'opening_day'])
   # for team in teams:
    for year in range (2000, 2018):
        try:
            df = pd.DataFrame(columns=['date', 'box', 'team', 'at', 'opponent', 'w_or_l', 'runs', 'runs_allowed',
                                       'innings', 'WL_record', 'rank', 'gb', 'winning_pitcher', 'losing_pitcher',
                                       'save', 'time', 'd_or_n', 'attendance', 'streak'])
            urls = requests.get('http://www.baseball-reference.com/teams/' + 'SEA' + '/' + str(year) + '-schedule-scores.shtml')
            soup = BeautifulSoup(urls.text, 'html.parser')
            for game in soup.find('table', {'class':'stats_table'}).find_all('tr'):
                results = []
                for element in game.find_all('td'):
                    results.append(element.text)
                    if len(results) == 19:
                        df.loc[len(df)] = results



            # double header variable for when 2 teams play same day in front of same crowd
            double_header = '('
            df['double_header'] = [1 if double_header in x else 0 for x in df.date]

            # convert date column to datetime
            df.date = df.date.str.replace(r"\(.*\)","")
            date = []
            for day in df.date:
                split = day.split(', ')
                date.append(split[1] + ', ' + str(year))
            df.date = date
            df.date = pd.to_datetime(df.date)

            # shift WL_record, rank, gb, streak and fill in the first value
            df.WL_record = df.WL_record.shift()
            df.WL_record.fillna('0-0', inplace=True)

            for col in ['rank', 'gb', 'streak']:
                df[col] = df[col].shift()
                df[col].fillna('0', inplace=True)

            # rolling means for runs and runs allowed
            df['runs_per_game'] = [0 if x == min(df.date) else df[df.date < x].runs.astype(int).mean() for x in df.date]
            df['runs_moving_avg'] = df.runs.rolling(5).mean().shift()
            df['runs_allowed_moving_avg'] = df.runs_allowed.rolling(5).mean().shift()
            df.runs_allowed_moving_avg.fillna(df.iloc[5].runs_allowed_moving_avg, inplace=True)

            # win_pct of last ten games
            win_pct = [list(df.w_or_l[i-10:i]).count('W') / 10. for i in range(10, len(df))]
            for i in range(10):
                win_pct.insert(0, None)
            df['win_pct'] = win_pct

            # return home games only
            df = df[~df['at'].str.contains('@')].reset_index(drop=True)

            # make dummny variable if game was on opening day or not
            df['opening_day'] = [1 if x == min(df.date) else 0 for x in df.date]

            # concatenate the two dataframes 
            team_df = pd.concat([team_df, df]).reset_index(drop=True)

            #sleep
            sleep(0.5)

        except:
            pass

    # save it to csv
    team_df.to_csv('/Users/andreewerner/Desktop/mlb-game-data-SEA.csv', index=False, encoding='utf-8')
    return team_df
games()

,date,box,team,at,opponent,w_or_l,runs,runs_allowed,innings,WL_record,rank,gb,winning_pitcher,losing_pitcher,save,time,d_or_n,attendance,streak,double_header,runs_per_game,runs_moving_avg,runs_allowed_moving_avg,win_pct,opening_day
0,2000-04-04,boxscore,SEA,,BOS,L,0,2,,0-0,0,0,Martinez,Moyer,Lowe,2:40,N,"45,552",0,0,0.000000,NaN,3.0,NaN,1
1,2000-04-05,boxscore,SEA,,BOS,W,9,3,,0-1,3,1.5,Garcia,Martinez,,3:11,N,"29,242",-,0,0.000000,NaN,3.0,NaN,0
2,2000-04-06,boxscore,SEA,,BOS,W,5,2,,1-1,3,0.5,Mesa,Florie,Sasaki,3:06,N,"25,121",+,0,4.500000,NaN,3.0,NaN,0
3,2000-04-07,boxscore,SEA,,NYY,W,7,5,,2-1,1,Tied,Halama,Pettitte,Sasaki,3:09,N,"40,827",++,0,4.666667,NaN,3.0,NaN,0
4,2000-04-08,boxscore,SEA,,NYY,L,2,3,,3-1,1,up 0.5,Nelson,Mesa,Rivera,3:35,D,"45,261",+++,0,5.250000,NaN,3.0,NaN,0
5,2000-04-09,boxscore,SEA,,NYY,W,9,3,,3-2,1,Tied,Moyer,Clemens,,2:40,D,"45,488",-,0,4.600000,4.6,3.0,NaN,0
6,2000-04-21,boxscore,SEA,,KCR,W,10,2,,8-6,1,up 0.5,Rodriguez,Durbin,,3:22,N,"37,034",--,0,7.000000,12.0,9.0,0.5,0
7,2000-04-22,boxscore,SEA,,KCR,W,4,2,,9-6,1,up 1.0,Tomko,Witasick,Sasaki,2:33,D,"38,200",+,0,7.200000,11.8,7.6,0.6,0
8,2000-04-23,boxscore,SEA,,KCR,W-wo,8,5,,10-6,1,up 2.0,Rhodes,Bottalico,,3:13,D,"29,701",++,0,7.000000,9.2,6.8,0.6,0
9,2000-04-24,boxscore,SEA,,CLE,L,0,6,,11-6,1,up 3.0,Finley,Meche,,2:42,N,"30,354",+++,0,7.058824,7.0,6.4,0.6,0


In [130]:
def games():
    team_df = pd.DataFrame(columns=['date', 'box', 'team', 'at', 'opponent', 'w_or_l', 'runs', 'runs_allowed',
                                    'innings', 'WL_record', 'rank', 'gb', 'winning_pitcher', 'losing_pitcher',
                                    'save', 'time', 'd_or_n', 'attendance', 'streak', 'double_header', 'runs_per_game',
                                    'runs_moving_avg', 'runs_allowed_moving_avg', 'win_pct', 'opening_day'])
   # for team in teams:
    for year in range (2000, 2018):
        try:
            df = pd.DataFrame(columns=['date', 'box', 'team', 'at', 'opponent', 'w_or_l', 'runs', 'runs_allowed',
                                       'innings', 'WL_record', 'rank', 'gb', 'winning_pitcher', 'losing_pitcher',
                                       'save', 'time', 'd_or_n', 'attendance', 'streak'])
            urls = requests.get('http://www.baseball-reference.com/teams/' + 'SFG' + '/' + str(year) + '-schedule-scores.shtml')
            soup = BeautifulSoup(urls.text, 'html.parser')
            for game in soup.find('table', {'class':'stats_table'}).find_all('tr'):
                results = []
                for element in game.find_all('td'):
                    results.append(element.text)
                    if len(results) == 19:
                        df.loc[len(df)] = results



            # double header variable for when 2 teams play same day in front of same crowd
            double_header = '('
            df['double_header'] = [1 if double_header in x else 0 for x in df.date]

            # convert date column to datetime
            df.date = df.date.str.replace(r"\(.*\)","")
            date = []
            for day in df.date:
                split = day.split(', ')
                date.append(split[1] + ', ' + str(year))
            df.date = date
            df.date = pd.to_datetime(df.date)

            # shift WL_record, rank, gb, streak and fill in the first value
            df.WL_record = df.WL_record.shift()
            df.WL_record.fillna('0-0', inplace=True)

            for col in ['rank', 'gb', 'streak']:
                df[col] = df[col].shift()
                df[col].fillna('0', inplace=True)

            # rolling means for runs and runs allowed
            df['runs_per_game'] = [0 if x == min(df.date) else df[df.date < x].runs.astype(int).mean() for x in df.date]
            df['runs_moving_avg'] = df.runs.rolling(5).mean().shift()
            df['runs_allowed_moving_avg'] = df.runs_allowed.rolling(5).mean().shift()
            df.runs_allowed_moving_avg.fillna(df.iloc[5].runs_allowed_moving_avg, inplace=True)

            # win_pct of last ten games
            win_pct = [list(df.w_or_l[i-10:i]).count('W') / 10. for i in range(10, len(df))]
            for i in range(10):
                win_pct.insert(0, None)
            df['win_pct'] = win_pct

            # return home games only
            df = df[~df['at'].str.contains('@')].reset_index(drop=True)

            # make dummny variable if game was on opening day or not
            df['opening_day'] = [1 if x == min(df.date) else 0 for x in df.date]

            # concatenate the two dataframes 
            team_df = pd.concat([team_df, df]).reset_index(drop=True)

            #sleep
            sleep(0.5)

        except:
            pass

    # save it to csv
    team_df.to_csv('/Users/andreewerner/Desktop/mlb-game-data-SFG.csv', index=False, encoding='utf-8')
    return team_df
games()

,date,box,team,at,opponent,w_or_l,runs,runs_allowed,innings,WL_record,rank,gb,winning_pitcher,losing_pitcher,save,time,d_or_n,attendance,streak,double_header,runs_per_game,runs_moving_avg,runs_allowed_moving_avg,win_pct,opening_day
0,2000-04-11,boxscore,SFG,,LAD,L,5,6,,3-4,4,2.5,Park,Rueter,Shaw,3:01,D,"40,930",--,0,5.142857,5.8,6.4,NaN,1
1,2000-04-12,boxscore,SFG,,LAD,L,5,6,,3-5,4,3.0,Adams,Weber,Shaw,3:18,N,"40,930",---,0,5.125000,4.6,5.8,NaN,0
2,2000-04-13,boxscore,SFG,,LAD,L,7,11,,3-6,5,3.0,Perez,Gardner,Shaw,3:11,N,"40,930",----,0,5.111111,4.8,6.0,NaN,0
3,2000-04-14,boxscore,SFG,,ARI,L,1,3,,3-7,5,4.0,Johnson,Hernandez,,2:32,N,"40,930",-----,0,5.300000,5.0,7.8,0.3,0
4,2000-04-15,boxscore,SFG,,ARI,L,4,7,,3-8,5,5.0,Stottlemyre,Ortiz,Swindell,3:07,D,"40,930",------,0,4.909091,4.2,7.0,0.3,0
5,2000-04-28,boxscore,SFG,,MON,L,3,9,,9-11,4,4.5,Telford,Ortiz,,3:07,N,"40,930",+++++,0,6.050000,9.0,5.8,0.6,0
6,2000-04-29,boxscore,SFG,,MON,W,2,1,,9-12,5,4.0,Johnstone,Telford,Nen,2:18,D,"40,930",-,0,5.904762,7.4,6.6,0.6,0
7,2000-04-30,boxscore,SFG,,MON,L,3,4,,10-12,3,4.0,Strickland,Johnstone,Urbina,2:52,D,"40,930",+,0,5.727273,6.2,5.6,0.7,0
8,2000-05-01,boxscore,SFG,,NYM,W,10,3,,10-13,3,5.0,Estes,Pulsipher,,2:50,N,"40,930",-,0,5.608696,4.4,5.0,0.6,0
9,2000-05-02,boxscore,SFG,,NYM,W,7,1,,11-13,3,4.5,Hernandez,Rusch,,2:17,N,"40,930",+,0,5.791667,5.2,4.8,0.7,0


In [131]:
def games():
    team_df = pd.DataFrame(columns=['date', 'box', 'team', 'at', 'opponent', 'w_or_l', 'runs', 'runs_allowed',
                                    'innings', 'WL_record', 'rank', 'gb', 'winning_pitcher', 'losing_pitcher',
                                    'save', 'time', 'd_or_n', 'attendance', 'streak', 'double_header', 'runs_per_game',
                                    'runs_moving_avg', 'runs_allowed_moving_avg', 'win_pct', 'opening_day'])
   # for team in teams:
    for year in range (2000, 2018):
        try:
            df = pd.DataFrame(columns=['date', 'box', 'team', 'at', 'opponent', 'w_or_l', 'runs', 'runs_allowed',
                                       'innings', 'WL_record', 'rank', 'gb', 'winning_pitcher', 'losing_pitcher',
                                       'save', 'time', 'd_or_n', 'attendance', 'streak'])
            urls = requests.get('http://www.baseball-reference.com/teams/' + 'STL' + '/' + str(year) + '-schedule-scores.shtml')
            soup = BeautifulSoup(urls.text, 'html.parser')
            for game in soup.find('table', {'class':'stats_table'}).find_all('tr'):
                results = []
                for element in game.find_all('td'):
                    results.append(element.text)
                    if len(results) == 19:
                        df.loc[len(df)] = results



            # double header variable for when 2 teams play same day in front of same crowd
            double_header = '('
            df['double_header'] = [1 if double_header in x else 0 for x in df.date]

            # convert date column to datetime
            df.date = df.date.str.replace(r"\(.*\)","")
            date = []
            for day in df.date:
                split = day.split(', ')
                date.append(split[1] + ', ' + str(year))
            df.date = date
            df.date = pd.to_datetime(df.date)

            # shift WL_record, rank, gb, streak and fill in the first value
            df.WL_record = df.WL_record.shift()
            df.WL_record.fillna('0-0', inplace=True)

            for col in ['rank', 'gb', 'streak']:
                df[col] = df[col].shift()
                df[col].fillna('0', inplace=True)

            # rolling means for runs and runs allowed
            df['runs_per_game'] = [0 if x == min(df.date) else df[df.date < x].runs.astype(int).mean() for x in df.date]
            df['runs_moving_avg'] = df.runs.rolling(5).mean().shift()
            df['runs_allowed_moving_avg'] = df.runs_allowed.rolling(5).mean().shift()
            df.runs_allowed_moving_avg.fillna(df.iloc[5].runs_allowed_moving_avg, inplace=True)

            # win_pct of last ten games
            win_pct = [list(df.w_or_l[i-10:i]).count('W') / 10. for i in range(10, len(df))]
            for i in range(10):
                win_pct.insert(0, None)
            df['win_pct'] = win_pct

            # return home games only
            df = df[~df['at'].str.contains('@')].reset_index(drop=True)

            # make dummny variable if game was on opening day or not
            df['opening_day'] = [1 if x == min(df.date) else 0 for x in df.date]

            # concatenate the two dataframes 
            team_df = pd.concat([team_df, df]).reset_index(drop=True)

            #sleep
            sleep(0.5)

        except:
            pass

    # save it to csv
    team_df.to_csv('/Users/andreewerner/Desktop/mlb-game-data-STL.csv', index=False, encoding='utf-8')
    return team_df
games()

,date,box,team,at,opponent,w_or_l,runs,runs_allowed,innings,WL_record,rank,gb,winning_pitcher,losing_pitcher,save,time,d_or_n,attendance,streak,double_header,runs_per_game,runs_moving_avg,runs_allowed_moving_avg,win_pct,opening_day
0,2000-04-03,boxscore,STL,,CHC,W,7,1,,0-0,0,0,Kile,Tapani,,2:54,D,"48,156",0,0,0.000000,NaN,5.0,NaN,1
1,2000-04-05,boxscore,STL,,CHC,W,10,4,,1-0,1,up 0.5,Hentgen,Lieber,,3:08,N,"38,908",+,0,7.000000,NaN,5.0,NaN,0
2,2000-04-06,boxscore,STL,,CHC,W,13,3,,2-0,1,Tied,Stephenson,Farnsworth,,2:48,D,"40,589",++,0,8.500000,NaN,5.0,NaN,0
3,2000-04-07,boxscore,STL,,MIL,L,1,9,,3-0,1,up 1.0,Bere,Benes,,3:25,N,"33,089",+++,0,10.000000,NaN,5.0,NaN,0
4,2000-04-08,boxscore,STL,,MIL,W,10,8,,3-1,1,Tied,Kile,Woodard,Veres,3:50,D,"40,909",-,0,7.750000,NaN,5.0,NaN,0
5,2000-04-09,boxscore,STL,,MIL,W,11,2,,4-1,1,up 1.0,Ankiel,De Los Santos,,3:03,D,"48,082",+,0,8.200000,8.2,5.0,NaN,0
6,2000-04-18,boxscore,STL,,SDP,W,5,4,,8-5,1,up 1.5,Benes,Meadows,Veres,2:48,N,"32,062",-,0,8.076923,7.0,8.4,0.5,0
7,2000-04-19,boxscore,STL,,SDP,W,4,3,,9-5,1,up 2.5,Kile,Hitchcock,Veres,2:41,N,"27,997",+,0,7.857143,7.0,7.8,0.6,0
8,2000-04-20,boxscore,STL,,SDP,W,14,1,,10-5,1,up 2.5,Ankiel,Boehringer,,3:30,N,"32,510",++,0,7.600000,6.6,6.0,0.6,0
9,2000-04-21,boxscore,STL,,COL,L,4,6,,11-5,1,up 2.5,Jarvis,Hentgen,Jimenez,3:18,N,"34,966",+++,0,8.000000,9.0,5.0,0.6,0


In [132]:
def games():
    team_df = pd.DataFrame(columns=['date', 'box', 'team', 'at', 'opponent', 'w_or_l', 'runs', 'runs_allowed',
                                    'innings', 'WL_record', 'rank', 'gb', 'winning_pitcher', 'losing_pitcher',
                                    'save', 'time', 'd_or_n', 'attendance', 'streak', 'double_header', 'runs_per_game',
                                    'runs_moving_avg', 'runs_allowed_moving_avg', 'win_pct', 'opening_day'])
   # for team in teams:
    for year in range (2000, 2018):
        try:
            df = pd.DataFrame(columns=['date', 'box', 'team', 'at', 'opponent', 'w_or_l', 'runs', 'runs_allowed',
                                       'innings', 'WL_record', 'rank', 'gb', 'winning_pitcher', 'losing_pitcher',
                                       'save', 'time', 'd_or_n', 'attendance', 'streak'])
            urls = requests.get('http://www.baseball-reference.com/teams/' + 'TBD' + '/' + str(year) + '-schedule-scores.shtml')
            soup = BeautifulSoup(urls.text, 'html.parser')
            for game in soup.find('table', {'class':'stats_table'}).find_all('tr'):
                results = []
                for element in game.find_all('td'):
                    results.append(element.text)
                    if len(results) == 19:
                        df.loc[len(df)] = results



            # double header variable for when 2 teams play same day in front of same crowd
            double_header = '('
            df['double_header'] = [1 if double_header in x else 0 for x in df.date]

            # convert date column to datetime
            df.date = df.date.str.replace(r"\(.*\)","")
            date = []
            for day in df.date:
                split = day.split(', ')
                date.append(split[1] + ', ' + str(year))
            df.date = date
            df.date = pd.to_datetime(df.date)

            # shift WL_record, rank, gb, streak and fill in the first value
            df.WL_record = df.WL_record.shift()
            df.WL_record.fillna('0-0', inplace=True)

            for col in ['rank', 'gb', 'streak']:
                df[col] = df[col].shift()
                df[col].fillna('0', inplace=True)

            # rolling means for runs and runs allowed
            df['runs_per_game'] = [0 if x == min(df.date) else df[df.date < x].runs.astype(int).mean() for x in df.date]
            df['runs_moving_avg'] = df.runs.rolling(5).mean().shift()
            df['runs_allowed_moving_avg'] = df.runs_allowed.rolling(5).mean().shift()
            df.runs_allowed_moving_avg.fillna(df.iloc[5].runs_allowed_moving_avg, inplace=True)

            # win_pct of last ten games
            win_pct = [list(df.w_or_l[i-10:i]).count('W') / 10. for i in range(10, len(df))]
            for i in range(10):
                win_pct.insert(0, None)
            df['win_pct'] = win_pct

            # return home games only
            df = df[~df['at'].str.contains('@')].reset_index(drop=True)

            # make dummny variable if game was on opening day or not
            df['opening_day'] = [1 if x == min(df.date) else 0 for x in df.date]

            # concatenate the two dataframes 
            team_df = pd.concat([team_df, df]).reset_index(drop=True)

            #sleep
            sleep(0.5)

        except:
            pass

    # save it to csv
    team_df.to_csv('/Users/andreewerner/Desktop/mlb-game-data-TBD.csv', index=False, encoding='utf-8')
    return team_df
games()

,date,box,team,at,opponent,w_or_l,runs,runs_allowed,innings,WL_record,rank,gb,winning_pitcher,losing_pitcher,save,time,d_or_n,attendance,streak,double_header,runs_per_game,runs_moving_avg,runs_allowed_moving_avg,win_pct,opening_day
0,2000-04-07,boxscore,TBD,,CLE,L,5,14,,2-2,3,0.5,Wright,Guzman,,3:22,N,"40,329",+,0,6.500000,NaN,7.2,NaN,1
1,2000-04-08,boxscore,TBD,,CLE,L,4,6,,2-3,3,1.5,Burba,Trachsel,Karsay,3:14,D,"30,177",-,0,6.200000,6.2,7.2,NaN,0
2,2000-04-09,boxscore,TBD,,CLE,L,4,17,,2-4,4,2.5,Colon,Rupe,,3:31,N,"21,357",--,0,5.833333,5.6,8.4,NaN,0
3,2000-04-11,boxscore,TBD,,CHW,L,6,13,,2-5,5,3.5,Parque,Yan,,2:58,N,"13,639",---,0,5.571429,5.4,10.6,NaN,0
4,2000-04-12,boxscore,TBD,,CHW,L,1,7,,2-6,5,3.5,Baldwin,Wheeler,,2:56,N,"14,034",----,0,5.625000,5.2,11.2,NaN,0
5,2000-04-13,boxscore,TBD,,CHW,W-wo,6,5,12,2-7,5,3.5,Mecir,Sturtze,,4:04,D,"15,464",-----,0,5.111111,4.0,11.4,NaN,0
6,2000-04-21,boxscore,TBD,,ANA,L,6,9,,5-10,5,6.5,Petkovsek,Hernandez,Percival,3:35,N,"22,221",--,0,5.133333,5.0,5.4,0.2,0
7,2000-04-22,boxscore,TBD,,ANA,W-wo,11,9,10,5-11,5,6.5,Lopez,Mercker,,3:25,D,"22,042",---,0,5.187500,5.2,5.2,0.2,0
8,2000-04-23,boxscore,TBD,,ANA,W,1,0,,6-11,5,5.5,Eiland,Dickson,Hernandez,2:41,D,"15,601",+,0,5.529412,6.0,7.0,0.2,0
9,2000-05-02,boxscore,TBD,,TEX,L,1,8,,9-15,5,6.5,Helling,Rupe,,3:14,N,"15,067",--,0,5.541667,6.4,4.8,0.3,0


In [133]:
def games():
    team_df = pd.DataFrame(columns=['date', 'box', 'team', 'at', 'opponent', 'w_or_l', 'runs', 'runs_allowed',
                                    'innings', 'WL_record', 'rank', 'gb', 'winning_pitcher', 'losing_pitcher',
                                    'save', 'time', 'd_or_n', 'attendance', 'streak', 'double_header', 'runs_per_game',
                                    'runs_moving_avg', 'runs_allowed_moving_avg', 'win_pct', 'opening_day'])
   # for team in teams:
    for year in range (2000, 2018):
        try:
            df = pd.DataFrame(columns=['date', 'box', 'team', 'at', 'opponent', 'w_or_l', 'runs', 'runs_allowed',
                                       'innings', 'WL_record', 'rank', 'gb', 'winning_pitcher', 'losing_pitcher',
                                       'save', 'time', 'd_or_n', 'attendance', 'streak'])
            urls = requests.get('http://www.baseball-reference.com/teams/' + 'TBR' + '/' + str(year) + '-schedule-scores.shtml')
            soup = BeautifulSoup(urls.text, 'html.parser')
            for game in soup.find('table', {'class':'stats_table'}).find_all('tr'):
                results = []
                for element in game.find_all('td'):
                    results.append(element.text)
                    if len(results) == 19:
                        df.loc[len(df)] = results



            # double header variable for when 2 teams play same day in front of same crowd
            double_header = '('
            df['double_header'] = [1 if double_header in x else 0 for x in df.date]

            # convert date column to datetime
            df.date = df.date.str.replace(r"\(.*\)","")
            date = []
            for day in df.date:
                split = day.split(', ')
                date.append(split[1] + ', ' + str(year))
            df.date = date
            df.date = pd.to_datetime(df.date)

            # shift WL_record, rank, gb, streak and fill in the first value
            df.WL_record = df.WL_record.shift()
            df.WL_record.fillna('0-0', inplace=True)

            for col in ['rank', 'gb', 'streak']:
                df[col] = df[col].shift()
                df[col].fillna('0', inplace=True)

            # rolling means for runs and runs allowed
            df['runs_per_game'] = [0 if x == min(df.date) else df[df.date < x].runs.astype(int).mean() for x in df.date]
            df['runs_moving_avg'] = df.runs.rolling(5).mean().shift()
            df['runs_allowed_moving_avg'] = df.runs_allowed.rolling(5).mean().shift()
            df.runs_allowed_moving_avg.fillna(df.iloc[5].runs_allowed_moving_avg, inplace=True)

            # win_pct of last ten games
            win_pct = [list(df.w_or_l[i-10:i]).count('W') / 10. for i in range(10, len(df))]
            for i in range(10):
                win_pct.insert(0, None)
            df['win_pct'] = win_pct

            # return home games only
            df = df[~df['at'].str.contains('@')].reset_index(drop=True)

            # make dummny variable if game was on opening day or not
            df['opening_day'] = [1 if x == min(df.date) else 0 for x in df.date]

            # concatenate the two dataframes 
            team_df = pd.concat([team_df, df]).reset_index(drop=True)

            #sleep
            sleep(0.5)

        except:
            pass

    # save it to csv
    team_df.to_csv('/Users/andreewerner/Desktop/mlb-game-data-TBR.csv', index=False, encoding='utf-8')
    return team_df
games()

,date,box,team,at,opponent,w_or_l,runs,runs_allowed,innings,WL_record,rank,gb,winning_pitcher,losing_pitcher,save,time,d_or_n,attendance,streak,double_header,runs_per_game,runs_moving_avg,runs_allowed_moving_avg,win_pct,opening_day
0,2008-04-08,boxscore,TBR,,SEA,L,5,6,,3-3,4,2.0,Bedard,Glover,Rowland-Smith,3:29,N,"36,048",--,0,5.333333,5.2,4.8,NaN,1
1,2008-04-09,boxscore,TBR,,SEA,L,1,7,,3-4,5,3.0,Washburn,Sonnanstine,,2:29,N,"12,106",---,0,5.285714,5.0,4.2,NaN,0
2,2008-04-10,boxscore,TBR,,SEA,W,7,0,,3-5,5,3.5,Jackson,Batista,,2:23,D,"11,898",----,0,4.750000,2.6,4.8,NaN,0
3,2008-04-11,boxscore,TBR,,BAL,W,10,5,,4-5,4,2.0,Reyes,Sarfate,,3:15,N,"12,146",+,0,5.000000,2.8,4.2,NaN,0
4,2008-04-12,boxscore,TBR,,BAL,L,2,3,,5-5,3,1.0,Bradford,Wheeler,Sherrill,2:51,N,"19,295",++,0,5.500000,4.8,4.8,0.5,0
5,2008-04-13,boxscore,TBR,,BAL,W,6,2,,5-6,5,2.0,Niemann,Burres,,2:50,D,"16,748",-,0,5.181818,5.0,4.2,0.4,0
6,2008-04-14,boxscore,TBR,,NYY,L,7,8,,6-6,4,1.0,Bruney,Reyes,Rivera,3:05,N,"18,872",+,0,5.250000,5.2,3.4,0.5,0
7,2008-04-15,boxscore,TBR,,NYY,L,3,5,,6-7,5,2.0,Pettitte,Jackson,Rivera,3:01,N,"20,923",-,0,5.384615,6.4,3.6,0.4,0
8,2008-04-18,boxscore,TBR,,CHW,L,2,9,,7-9,5,2.5,Vazquez,Niemann,,3:02,N,"12,379",+,0,5.312500,5.6,4.8,0.4,0
9,2008-04-19,boxscore,TBR,,CHW,W,5,0,,7-10,5,3.5,Sonnanstine,Buehrle,,2:02,N,"17,613",-,0,5.117647,4.8,6.2,0.4,0


In [134]:
def games():
    team_df = pd.DataFrame(columns=['date', 'box', 'team', 'at', 'opponent', 'w_or_l', 'runs', 'runs_allowed',
                                    'innings', 'WL_record', 'rank', 'gb', 'winning_pitcher', 'losing_pitcher',
                                    'save', 'time', 'd_or_n', 'attendance', 'streak', 'double_header', 'runs_per_game',
                                    'runs_moving_avg', 'runs_allowed_moving_avg', 'win_pct', 'opening_day'])
   # for team in teams:
    for year in range (2000, 2018):
        try:
            df = pd.DataFrame(columns=['date', 'box', 'team', 'at', 'opponent', 'w_or_l', 'runs', 'runs_allowed',
                                       'innings', 'WL_record', 'rank', 'gb', 'winning_pitcher', 'losing_pitcher',
                                       'save', 'time', 'd_or_n', 'attendance', 'streak'])
            urls = requests.get('http://www.baseball-reference.com/teams/' + 'TEX' + '/' + str(year) + '-schedule-scores.shtml')
            soup = BeautifulSoup(urls.text, 'html.parser')
            for game in soup.find('table', {'class':'stats_table'}).find_all('tr'):
                results = []
                for element in game.find_all('td'):
                    results.append(element.text)
                    if len(results) == 19:
                        df.loc[len(df)] = results



            # double header variable for when 2 teams play same day in front of same crowd
            double_header = '('
            df['double_header'] = [1 if double_header in x else 0 for x in df.date]

            # convert date column to datetime
            df.date = df.date.str.replace(r"\(.*\)","")
            date = []
            for day in df.date:
                split = day.split(', ')
                date.append(split[1] + ', ' + str(year))
            df.date = date
            df.date = pd.to_datetime(df.date)

            # shift WL_record, rank, gb, streak and fill in the first value
            df.WL_record = df.WL_record.shift()
            df.WL_record.fillna('0-0', inplace=True)

            for col in ['rank', 'gb', 'streak']:
                df[col] = df[col].shift()
                df[col].fillna('0', inplace=True)

            # rolling means for runs and runs allowed
            df['runs_per_game'] = [0 if x == min(df.date) else df[df.date < x].runs.astype(int).mean() for x in df.date]
            df['runs_moving_avg'] = df.runs.rolling(5).mean().shift()
            df['runs_allowed_moving_avg'] = df.runs_allowed.rolling(5).mean().shift()
            df.runs_allowed_moving_avg.fillna(df.iloc[5].runs_allowed_moving_avg, inplace=True)

            # win_pct of last ten games
            win_pct = [list(df.w_or_l[i-10:i]).count('W') / 10. for i in range(10, len(df))]
            for i in range(10):
                win_pct.insert(0, None)
            df['win_pct'] = win_pct

            # return home games only
            df = df[~df['at'].str.contains('@')].reset_index(drop=True)

            # make dummny variable if game was on opening day or not
            df['opening_day'] = [1 if x == min(df.date) else 0 for x in df.date]

            # concatenate the two dataframes 
            team_df = pd.concat([team_df, df]).reset_index(drop=True)

            #sleep
            sleep(0.5)

        except:
            pass

    # save it to csv
    team_df.to_csv('/Users/andreewerner/Desktop/mlb-game-data-TEX.csv', index=False, encoding='utf-8')
    return team_df
games()


,date,box,team,at,opponent,w_or_l,runs,runs_allowed,innings,WL_record,rank,gb,winning_pitcher,losing_pitcher,save,time,d_or_n,attendance,streak,double_header,runs_per_game,runs_moving_avg,runs_allowed_moving_avg,win_pct,opening_day
0,2000-04-03,boxscore,TEX,,CHW,W,10,4,,0-0,0,0,Rogers,Sirotka,,2:39,D,"49,332",0,0,0.000000,NaN,7.0,NaN,1
1,2000-04-04,boxscore,TEX,,CHW,W,12,8,,1-0,1,up 0.5,Cordero,Simas,,3:47,N,"29,418",+,0,10.000000,NaN,7.0,NaN,0
2,2000-04-05,boxscore,TEX,,CHW,L,8,12,,2-0,1,up 1.0,Foulke,Zimmerman,,3:30,D,"25,544",++,0,11.000000,NaN,7.0,NaN,0
3,2000-04-06,boxscore,TEX,,CHW,L,2,6,,2-1,1,Tied,Baldwin,Loaiza,,2:23,N,"28,466",-,0,10.000000,NaN,7.0,NaN,0
4,2000-04-07,boxscore,TEX,,TOR,W,11,5,,2-2,3,0.5,Clark,Castillo,,2:49,N,"31,619",--,0,8.000000,NaN,7.0,NaN,0
5,2000-04-08,boxscore,TEX,,TOR,L,0,4,,3-2,2,0.5,Wells,Rogers,,2:52,N,"37,128",+,0,8.600000,8.6,7.0,NaN,0
6,2000-04-09,boxscore,TEX,,TOR,W,7,5,,3-3,3,0.5,Helling,Halladay,,3:01,D,"40,186",-,0,7.166667,6.6,7.0,NaN,0
7,2000-04-17,boxscore,TEX,,NYY,L,4,5,11,6-6,3,2.0,Rivera,Crabtree,Erdos,3:52,N,"38,166",-,0,5.916667,4.2,4.2,0.4,0
8,2000-04-18,boxscore,TEX,,NYY,L,3,6,,6-7,3,2.5,Hernandez,Rogers,,2:50,N,"43,644",--,0,5.769231,3.8,3.6,0.4,0
9,2000-04-19,boxscore,TEX,,NYY,L,4,5,10,6-8,3,2.5,Rivera,Zimmerman,,3:22,D,"39,294",---,0,5.571429,4.2,3.8,0.4,0


In [135]:
def games():
    team_df = pd.DataFrame(columns=['date', 'box', 'team', 'at', 'opponent', 'w_or_l', 'runs', 'runs_allowed',
                                    'innings', 'WL_record', 'rank', 'gb', 'winning_pitcher', 'losing_pitcher',
                                    'save', 'time', 'd_or_n', 'attendance', 'streak', 'double_header', 'runs_per_game',
                                    'runs_moving_avg', 'runs_allowed_moving_avg', 'win_pct', 'opening_day'])
   # for team in teams:
    for year in range (2000, 2018):
        try:
            df = pd.DataFrame(columns=['date', 'box', 'team', 'at', 'opponent', 'w_or_l', 'runs', 'runs_allowed',
                                       'innings', 'WL_record', 'rank', 'gb', 'winning_pitcher', 'losing_pitcher',
                                       'save', 'time', 'd_or_n', 'attendance', 'streak'])
            urls = requests.get('http://www.baseball-reference.com/teams/' + 'TOR' + '/' + str(year) + '-schedule-scores.shtml')
            soup = BeautifulSoup(urls.text, 'html.parser')
            for game in soup.find('table', {'class':'stats_table'}).find_all('tr'):
                results = []
                for element in game.find_all('td'):
                    results.append(element.text)
                    if len(results) == 19:
                        df.loc[len(df)] = results



            # double header variable for when 2 teams play same day in front of same crowd
            double_header = '('
            df['double_header'] = [1 if double_header in x else 0 for x in df.date]

            # convert date column to datetime
            df.date = df.date.str.replace(r"\(.*\)","")
            date = []
            for day in df.date:
                split = day.split(', ')
                date.append(split[1] + ', ' + str(year))
            df.date = date
            df.date = pd.to_datetime(df.date)

            # shift WL_record, rank, gb, streak and fill in the first value
            df.WL_record = df.WL_record.shift()
            df.WL_record.fillna('0-0', inplace=True)

            for col in ['rank', 'gb', 'streak']:
                df[col] = df[col].shift()
                df[col].fillna('0', inplace=True)

            # rolling means for runs and runs allowed
            df['runs_per_game'] = [0 if x == min(df.date) else df[df.date < x].runs.astype(int).mean() for x in df.date]
            df['runs_moving_avg'] = df.runs.rolling(5).mean().shift()
            df['runs_allowed_moving_avg'] = df.runs_allowed.rolling(5).mean().shift()
            df.runs_allowed_moving_avg.fillna(df.iloc[5].runs_allowed_moving_avg, inplace=True)

            # win_pct of last ten games
            win_pct = [list(df.w_or_l[i-10:i]).count('W') / 10. for i in range(10, len(df))]
            for i in range(10):
                win_pct.insert(0, None)
            df['win_pct'] = win_pct

            # return home games only
            df = df[~df['at'].str.contains('@')].reset_index(drop=True)

            # make dummny variable if game was on opening day or not
            df['opening_day'] = [1 if x == min(df.date) else 0 for x in df.date]

            # concatenate the two dataframes 
            team_df = pd.concat([team_df, df]).reset_index(drop=True)

            #sleep
            sleep(0.5)

        except:
            pass

    # save it to csv
    team_df.to_csv('/Users/andreewerner/Desktop/mlb-game-data-TOR.csv', index=False, encoding='utf-8')
    return team_df
games()

,date,box,team,at,opponent,w_or_l,runs,runs_allowed,innings,WL_record,rank,gb,winning_pitcher,losing_pitcher,save,time,d_or_n,attendance,streak,double_header,runs_per_game,runs_moving_avg,runs_allowed_moving_avg,win_pct,opening_day
0,2000-04-03,boxscore,TOR,,KCR,W-wo,5,4,,0-0,0,0,Koch,Spradlin,,2:46,D,"40,898",0,0,0.000000,NaN,6.2,NaN,1
1,2000-04-04,boxscore,TOR,,KCR,W,6,3,,1-0,1,Tied,Halladay,Witasick,Koch,2:33,N,"13,514",+,0,5.000000,NaN,6.2,NaN,0
2,2000-04-05,boxscore,TOR,,KCR,L,3,4,,2-0,1,Tied,Rosado,Carpenter,Bottalico,2:45,N,"14,957",++,0,5.500000,NaN,6.2,NaN,0
3,2000-04-06,boxscore,TOR,,KCR,L,3,9,,2-1,1,Tied,Durbin,Escobar,Rigby,3:00,D,"14,336",-,0,4.666667,NaN,6.2,NaN,0
4,2000-04-14,boxscore,TOR,,SEA,L,9,11,,4-6,4,2.0,Moyer,Wells,,3:06,N,"17,306",+,0,4.100000,3.8,4.0,0.3,0
5,2000-04-15,boxscore,TOR,,SEA,L,6,17,,4-7,4,3.0,Rodriguez,Halladay,Paniagua,3:22,D,"21,754",-,0,4.545455,4.8,6.2,0.3,0
6,2000-04-16,boxscore,TOR,,SEA,L,7,19,,4-8,4,4.0,Garcia,Carpenter,,3:24,D,"15,325",--,0,4.666667,5.0,8.2,0.2,0
7,2000-04-17,boxscore,TOR,,ANA,W,7,1,,4-9,5,5.0,Escobar,Ortiz,,3:03,N,"13,622",---,0,4.846154,6.4,10.8,0.2,0
8,2000-04-18,boxscore,TOR,,ANA,L,10,16,,5-9,5,5.0,Dickson,Castillo,,3:51,N,"13,825",+,0,5.000000,7.0,10.0,0.3,0
9,2000-04-19,boxscore,TOR,,ANA,W,12,4,,5-10,5,6.0,Wells,Hill,,2:45,N,"13,572",-,0,5.333333,7.8,12.8,0.3,0


In [ ]:
def games():
    team_df = pd.DataFrame(columns=['date', 'box', 'team', 'at', 'opponent', 'w_or_l', 'runs', 'runs_allowed',
                                    'innings', 'WL_record', 'rank', 'gb', 'winning_pitcher', 'losing_pitcher',
                                    'save', 'time', 'd_or_n', 'attendance', 'streak', 'double_header', 'runs_per_game',
                                    'runs_moving_avg', 'runs_allowed_moving_avg', 'win_pct', 'opening_day'])
   # for team in teams:
    for year in range (2000, 2018):
        try:
            df = pd.DataFrame(columns=['date', 'box', 'team', 'at', 'opponent', 'w_or_l', 'runs', 'runs_allowed',
                                       'innings', 'WL_record', 'rank', 'gb', 'winning_pitcher', 'losing_pitcher',
                                       'save', 'time', 'd_or_n', 'attendance', 'streak'])
            urls = requests.get('http://www.baseball-reference.com/teams/' + 'WSN' + '/' + str(year) + '-schedule-scores.shtml')
            soup = BeautifulSoup(urls.text, 'html.parser')
            for game in soup.find('table', {'class':'stats_table'}).find_all('tr'):
                results = []
                for element in game.find_all('td'):
                    results.append(element.text)
                    if len(results) == 19:
                        df.loc[len(df)] = results



            # double header variable for when 2 teams play same day in front of same crowd
            double_header = '('
            df['double_header'] = [1 if double_header in x else 0 for x in df.date]

            # convert date column to datetime
            df.date = df.date.str.replace(r"\(.*\)","")
            date = []
            for day in df.date:
                split = day.split(', ')
                date.append(split[1] + ', ' + str(year))
            df.date = date
            df.date = pd.to_datetime(df.date)

            # shift WL_record, rank, gb, streak and fill in the first value
            df.WL_record = df.WL_record.shift()
            df.WL_record.fillna('0-0', inplace=True)

            for col in ['rank', 'gb', 'streak']:
                df[col] = df[col].shift()
                df[col].fillna('0', inplace=True)

            # rolling means for runs and runs allowed
            df['runs_per_game'] = [0 if x == min(df.date) else df[df.date < x].runs.astype(int).mean() for x in df.date]
            df['runs_moving_avg'] = df.runs.rolling(5).mean().shift()
            df['runs_allowed_moving_avg'] = df.runs_allowed.rolling(5).mean().shift()
            df.runs_allowed_moving_avg.fillna(df.iloc[5].runs_allowed_moving_avg, inplace=True)

            # win_pct of last ten games
            win_pct = [list(df.w_or_l[i-10:i]).count('W') / 10. for i in range(10, len(df))]
            for i in range(10):
                win_pct.insert(0, None)
            df['win_pct'] = win_pct

            # return home games only
            df = df[~df['at'].str.contains('@')].reset_index(drop=True)

            # make dummny variable if game was on opening day or not
            df['opening_day'] = [1 if x == min(df.date) else 0 for x in df.date]

            # concatenate the two dataframes 
            team_df = pd.concat([team_df, df]).reset_index(drop=True)

            #sleep
            sleep(0.5)

        except:
            pass

    # save it to csv
    team_df.to_csv('/Users/andreewerner/Desktop/mlb-game-data-WSN.csv', index=False, encoding='utf-8')
    return team_df
games()